**Notes**:

 - Take a look at the [Exchanging Data With an Excel Worksheet](#Exchanging-Data-With-an-Excel-Worksheet);  What to do about the functions? Define the three in the chapter? `XLPut`, `XLGet` and `XLConver` or should I provide the utility `.DWS`? Or should I move these 3 to a smaller `.DWS`?
 - in the same section, should I change the functions to ask for user input / give error messages as standard IO instead of using the fancy GUI things? Those work poorly with the Jupyter notebook and I'm afraid they may not work on other platforms.
 - write about tail recursion in specialist's section;
 - This chapter uses some helper files that should be linked to from this chapter. I can't find them online, where are they? Should I upload them to somewhere else and link to that place? Should I upload them to GH, for example?
 - the book says "The SQL interface is not described in this tutorial; please refer instead to the specialised brochures.". What brochures? Where are they?
 - **PRINTING STUFF ONLY WORKS ON WINDOWS, PROBABLY... BUT REPLACE IT WITH BUILDING A PDF AND PRINTING IT LATER.** does the stuff on "Printing Results on a Printer" work on mac/Linux? And is this section relevant, at all..? also this section talks about newleaf and rainpro, formerly in chapter S. Are newleaf and rainpro still relevant and will they be kept in this new version of the book?
 - **CHECK THE `CALC` WS** on the section about using GUIs for I/O it has a screenshot. Can I get a screenshot of a GUI of a more modern APL program?
 - **KEEP THEM AS IS; WHEN IT MAKES SENSE, ASK TO DO THE EXERCISE WITH A DFN AS WELL** there's many exercises targeting tradfns and older control structures. Should I leave those as-is? Adapt them? Replace them with exercises more suitable for dfns? Truth be told, with the little primitives the user knows and no operators at all, it isn't very easy to write many useful functions.

# User Defined Functions

## Landmarks

### Some Definitions

In previous chapters we made a distinction between the functions and operators which are part of APL, like `+`, `×`, `⌈` and `/` (we refer to them as *primitives*), and those functions and operators that are created by the user which are represented, not by a symbol, but by a name like `Average` (we say they are *user-defined*).

We also made an important distinction between *functions*, which apply to data and which return data, and *operators*, which apply to functions or data and produce *derived* functions (see [the definition of Reduce](./Some-Primitive-Functions.ipynb#Definition)).

This means that we can distinguish between 4 major categories of processing tools:

| Category | Name | Examples | Refer to |
| :- | :- | :-: | :-: |
| Built-in tools | Primitive functions | `+` `×` `⌈` `⍴` | Previous chapters |
| | Primitive operators | `/` | [Chapter on Operators](./Operators.ipynb) |
| User-defined tools | **User-defined functions** | `Average` | This chapter |
| | User-defined operators | | [Section on User-Defined Operators](./Operators.ipynb#User-Defined-Operators) |

This chapter is devoted to user-defined *functions*. The subject of user-defined *operators* will be covered later.

We can further categorise user-defined functions according to the way they process data. Firstly we can distinguish between ***Direct*** and ***Procedural*** **functions**.

 - ***Direct*** functions (commonly referred to as ***dfns***) are defined in a very formal manner.
   - They are usually designed for pure calculation, without any external or user interfaces. *Dfns* do not allow loops except by recursion and have limited options for conditional programming.
   
 - ***Procedural*** functions (commonly referred to as ***tradfns***, short for *traditional functions*) are less formal and look much more like programs written in other languages.
   - They provide greater flexibility for building major applications which involve user interfaces, access to files or databases and other external interfaces. *Tradfns* may take no arguments and behave like scripts.
   
Even though you may write entire systems with dfns, you might prefer to restrict their use to encapsulate statements that, together, perform some meaningful operation on the data given.

The second distinction we can make concerns the number of arguments a user-defined function can have.

 - ***Dyadic*** functions take two arguments which are placed on either side of the function (`X f Y`);
 - ***Monadic*** functions take a single argument which is placed to the right of the function (`f Y`);
 - ***Niladic*** functions take no argument at all;
 - ***Ambivalent*** functions are dyadic functions whose left argument is optional.

### Configure Your Environment

Dyalog APL has a highly configurable development and debugging environment, designed to fit the requirements of very different kinds of programmers. This environment is controlled by configuration parameters; let us determine which context will suit you best.

#### What Do You Need?

All you need (except for love) is:

 - a window in which to type expressions that you want to be executed (white Session window);
 - one or more windows in which to create/modify user-defined functions (grey Edit windows);
 - one or more windows to debug execution errors (black Trace window).
 
The colours above refer to the positions of the windows in <!--figure-->the figures below<!--IDE_Window_Layout,IDE_Window_Horizontal_Layout,IDE_Floating_Window_Layout-->.

The default configuration is consistent with other software development tools and in it is possible to divide the session window into three parts which can be resized, as shown in <!--figure-->here<!--IDE_Window_Layout-->:

![The default window configuration](res/IDE_Window_Layout.png)

This configuration provides a single Edit window and a single Trace window, each of which is "docked" along one of the Session window borders. You can dock these windows along any of the Session window sides. For example, <!--figure-->the figure below<!--IDE_Window_Horizontal_Layout--> shows a configuration with three horizontal panes, highly suitable for entering and editing very long statements.

![The Edit and Trace windows in a horizontal layout](res/IDE_Window_Horizontal_Layout.png)

The Edit window supports the *Multiple Document Interface* (MDI). This means that you can work on more than one function at a time.

 - On the Windows interpreter you can use the "Window" menu to *Tile* and *Cascade*, or you can maximise any one of the functions to concentrate solely upon it.
 - If you are using RIDE the default behaviour is to open a tab per item you are editing.
 
If you are working on a relatively small screen you may find that you prefer to work with "floating" windows in a layout similar to the one in <!--figure-->here<!--IDE_Floating_Window_Layout-->:

!["floating" windows layout](res/IDE_Floating_Window_Layout.png)

 - On the Windows interpreter, you can either
   - grab the border of a sub-window (Edit or Trace) and then drag and drop it in the middle of the session window, as an independent floating window,
   - or enable the "Classic Dyalog mode", which can be set under "Options" ⇨ "Configure..." ⇨ "Trace/Edit" as shown in <!--figure-->the figure below<!--IDE_Configure_Classic_Mode-->.

![Option to set "Classic Dyalog mode"](res/IDE_Configure_Classic_Mode.png)

 - If you are using RIDE you can go to "Edit" ⇨ "Preferences" ⇨ "Windows" and enable "Floating windows" as shown in <!--figure-->the next figure<!--RIDE_Configure_Floating_Windows-->:
 
![Enable "floating" windows in RIDE](res/RIDE_Configure_Floating_Windows.png)

Working with floating windows has the added benefit of allowing you to have a stack of trace windows (as opposed to a single trace window), showing which functions call which other. This will be explored in [the section on configuring the trace tools](./First-Aid-Kit.ipynb#Choose-Your-Configuration).

#### A Text Editor; What For?

Some dfns can be defined by a single expression and so are easy to define inside the session. We used this technique before to define a function named `Average` and here we do it again:

In [1]:
Average ← {(+/⍵)÷≢⍵}

However, as one defines more complex functions, it can become more complicated to define dfns in the session window.

For one, the ability to define multi-line dfns in the session was only made available with Dyalog 18.0. In these notebooks you can see that multi-line dfns are defined in cells that start with `]dinput`:

In [2]:
]dinput
Average ← {
    (+/⍵)÷≢⍵
}

In the Windows interpreter an expression like the one above might result in a `SYNTAX ERROR`, as you type `Average ← {`, then hit <kbd>Enter</kbd> to change line, and then the interpreter tries to execute the line you entered, instead of allowing you to continue the definition of `Average`. To change this behaviour and allow for multi-line dfns, you can go to "Options" &#8680; "Configure..." &#8680; "Session" and check the experimental multi-line input box at the bottom.

If you are using RIDE and this capability is not ON by default, you can turn it ON by setting the `DYALOG_LINEEDITOR_MODE` environment variable to `1` in the connection menu, like demonstrated in <!--figure-->the next figure<!--RIDE_Enable_Multiline_Input-->.

![Configuring RIDE to allow for multi-line input](res/RIDE_Enable_Multiline_Input.png)

Secondly, even if multi-line input is enabled, you have to be very careful not to press <kbd>Enter</kbd> and ***fix*** the dfn by mistake. This is not a terrible blunder but it does lose some time and breaks your train of thought. In APL, to ***fix*** a file means to define the variables (data, functions and operators) in the file and to make them available in the session. Hence, to fix a function means assigning it to a name and making it available for use in the session window.

For these reasons, it is more appropriate to edit multi-line dfns and tradfns in a suitable text editor. The built-in editors for the Windows interpreter and for RIDE are likely to be suitable for you, but other alternatives exist. You can find an enumeration of most of the available alternatives over at [the APL Wiki](https://aplwiki.com/wiki/Text_editors). We will also cover this in more depth in [the chapter about source code management](./Source-Code-Management.ipynb).

## Simple Dfns

### Definition

Dfns are a set of statements enclosed by curly braces `{}`, so a simple dfn is typically created with the syntax `Name ← { definition }` where

 - `Name` is the function name. It is followed by a definition, delimited by a pair of curly braces `{` and `}`. This definition may make use of one or two variables named `⍵` and `⍺`, which represent the values to be processed. `⍵` and `⍺` are called ***arguments*** of the function;
 - `⍵` (<kbd>APL</kbd>+<kbd>w</kbd>) is a generic symbol which represents the right argument of the function;
 - `⍺` (<kbd>APL</kbd>+<kbd>a</kbd>) is a generic symbol which represents the left argument if the function is dyadic.
 
Here is an example monadic dfn:

In [3]:
Average ← {(+/⍵)÷(≢⍵)}

And here are two more dyadic dfns, and an example showing how they can be used:

In [4]:
Plus ← {⍺ + ⍵}
Times ← {⍺ × ⍵}
3 Times 7 Plus 9

48

Notice the final statement above is strictly equivalent to

In [5]:
3 × 7 + 9

48

as the order of evaluation is the same.

The arguments `⍵` and `⍺` are read-only (they cannot be reassigned) and are limited in scope to only being visible within the function itself. The only exception to the "read-only rule" is when providing a default left argument to the dfn, which we'll cover in [a bit](#Default-Left-Argument).

The developer does not need to declare anything about the shape or internal representation of the arguments and the result. This information is automatically obtained from the arrays provided as its arguments. This is similar to the behaviour of dynamically typed programming languages, such as Python or Javascript. So, our functions can work on any arrays.

A scalar added to a matrix returns a matrix. No need to specify it:

In [6]:
12 Plus 2 3⍴⍳6

13 14 15
16 17 18

And a vector of integer numbers multiplied by a scalar fractional number returns a vector of fractional numbers:

In [7]:
7.3 Times 10 34 52 16

73 248.2 379.6 116.8

These simple dfns are well suited for pure calculations of straightforward array manipulation. For example, here is how we can calculate the hypotenuse of a right-angled triangle from the lengths of the two other sides:

In [8]:
Hypo ← {(+/⍵*2)*0.5}
Hypo 3 4

5

In [9]:
Hypo 12 5

13

### Unnamed Dfns

A dfn can be defined and then discarded immediately after it has been used, in which case it does not need a name. For example, the geometric mean of a set of $N$ values is defined as the $N^\text{th}$ root of their product. The function can be defined and used inline like this:

In [10]:
{(×/⍵)*÷≢⍵} 6 8 4 11 9 14 7

7.8657

But because we didn't assign it to a name, it was discarded after being used and can't be used again. This kind of function is similar to *inline* or *lambda* functions in other languages.

A special case is `{}`. This function does nothing but placed at the left of an expression, it can be used to prevent the result of the expression from being displayed on the screen:

In [11]:
3 Plus 3

6

In [12]:
{} 3 Plus 3

### Modifying The Code

Single line dfns may be modified using the function editor, as will be explained in the next section. They can also be redefined entirely, as many times as necessary, as shown:

In [13]:
Magic ← {⍺+⍵}

In [14]:
Magic ← {⍺÷+/⍵}

In [15]:
Magic ← {(+/⍺)-(+/⍵)}

We defined `Magic` and then changed it twice. Only the most recent definition will survive.

Now we will delve into how to define and use more complex dfns. For this we will explore the built-in editor that comes with your interpreter and you will also learn some more syntax to empower your dfns.

## More on Dfns

The dfns we wrote in the previous section were very simple and consisted of a single statement. We will now use the text editor to define multi-line dfns.

### Characteristics

 - Generally, the opening and closing braces are placed alone on the first and last lines. This is not mandatory, it is just a convention;
 - dfns might be commented at will;
 - one can create as many variables as needed: they are automatically deemed to be local variables, which means they only exist while inside the dfn. Note that this is opposite to the default behaviour of tradfns, where all variables are global (cf. [this subsection on dfns](#Local-Variables) and [this subsection on tradfns](#The-Default-Isn't-Local));
 - the arguments `⍵` and `⍺` retain the values passed to them as arguments and may not be changed. Any attempt to modify them causes a `SYNTAX ERROR` to be reported, except when *defaulting the left argument* (cf. [the subsection on "Default Left Argument"](#Default-Left-Argument));
 - as soon as an expression generates a result that is not assigned to a name or used in any other way, the function terminates, and the value of that expression is returned as the result of the function. If the function contains more lines they will not be executed (cf. [the subsection on "Returning the Result"](#Returning-the-Result));
 - traditional *control structures* and *branching* cannot be used in dfns (cf. [the subsection on "Guards"](#Guards)).
 
We will explore these characteristics in the following sections.

### A Working Example

As an example, let us see how we could define a function to calculate the *harmonic mean* of a vector of numbers. The harmonic mean of a vector is the inverse of the sum of the inverses of the numbers in the vector. This will become clearer when you see the code.

First of all, we must choose a name for our new function. We will choose to name it `HarmonicMean`.

Among the multiple ways of invoking the text editor, let us use a very simple one: type the command `)ED` followed by a space and the name of the function to create: `)ED HarmonicMean`.

Unless you already redefined the defaults, a mostly empty window should open to the right of the session. We include an example screenshot of such a window from the Windows interpreter in <!--figure-->the figure below<!--Win_Editor-->:

![The edit window opened by the command `)ED HarmonicMean`](res/Win_Editor.png)

Now that we have a window in which to define our dfn, we can go ahead and implement the harmonic mean. We shall split the process into a series of simple steps, as shown in <!--figure-->the next figure<!--Win_Editor_Harmonic_Mean-->: calculate the inverses, sum them, invert the sum.

![The `HarmonicMean` function defined in the edit window](res/Win_Editor_Harmonic_Mean.png)

We also define it in this notebook:

In [16]:
]dinput
HarmonicMean ← {
    inverses ← ÷⍵
    sum ← +/inverses
    ÷sum
}

Now that we know how to compute the harmonic mean of a vector of numbers, we just have to ***fix*** it so that we can use it in our session. ***Fixing*** a function is somewhat analogous to compilation in other programming languages, but can also be seen as a sort of "File save" followed by an "import" in interpreted languages. There are many ways to fix a function:

| Interpreter | *Fix* method |
| :- | :- |
| Windows interpreter | go to "File" &#8680; "Fix" |
| RIDE | right-click the edit window &#8680; "Fix" |
| both | press <kbd>Esc</kbd> (also closes the edit window) |
| both | define a custom keyboard shortcut |

Following one of the appropriate methods should make the `HarmonicMean` function available for use in the session window. You can make sure it worked by simply typing in the name of the function and pressing <kbd>Enter</kbd> in the session:

In [17]:
HarmonicMean

{ 
 inverses←÷⍵ 
 sum←+/inverses
 ÷sum 
 }

If instead of getting the source code of the function you get a `VALUE ERROR`, then the function wasn't properly fixed.

Now that we can compute the harmonic mean of a vector of numbers, we can answer questions like the following:

"*If I take 6 hours to paint a wall and you take 2 hours, how much time will we need to paint the wall if we do it together?*"

This type of question can be answered by taking the harmonic mean of the individual times:

In [18]:
HarmonicMean 6 2

1.5

So the two of us would take 1h30min to paint the whole wall.

Similarly, if we had further help from two people who could paint the whole wall in 4 and 5 hours, respectively, the four of us would need

In [19]:
times ← 6 2 4 5
⊢hours ← HarmonicMean times

0.895522

hours, or approximately

In [20]:
⌊60×hours

53

minutes.

After using your function for a bit you realise it is over-complicated, in the sense that it involves too many intermediate steps and you wish to get rid of those. If your edit window is still open, you can simply edit the function and fix it again. If the edit window was closed, you can type `)ED HarmonicMean` again or you can double-click the name `HarmonicMean` in the session. Both options will open the appropriate edit window.

After having done so, perhaps you rewrite your function to

In [21]:
]dinput
HarmonicMean ← {
    inverses ← ÷⍵
    ÷+/inverses
}

Then you fix it and use it again a couple of times:

In [22]:
HarmonicMean times

0.895522

In [23]:
HarmonicMean 4 3 2 1 0

DOMAIN ERROR: Divide by zero
HarmonicMean[1] inverses←÷⍵
                         ∧


But now it resulted in an error, and the error messages says `HarmonicMean[1] inverses←÷⍵`. This `HarmonicMean[1]` means the error was in line 1 of the `HarmonicMean` function. Right next to it, it also shows the part of the code that caused the error, but supposed you had a really long file, perhaps with multiple functions. How would you find the appropriate line in the first place?

Thankfully, both RIDE and the Windows interpreter have an option that can be set to display line numbers (cf. <!--figure-->the figure below<!--Editors_Toggle_Line_Numbers-->).

![Option to toggle line numbers in RIDE (to the right) and the Windows interpreter (to the left)](res/Editors_Toggle_Line_Numbers.png)



Different people like to comment their code in different ways, and naturally dfns can be commented. For illustrative purposes, consider the dfn that follows, which has comments before any statement, inline with some statements, between the statements and at the end of the dfn:

In [24]:
]dinput
HarmonicMean ← {
    ⍝ Monadic function to compute the harmonic mean of a vector
    inverses ← ÷⍵   ⍝ This inverts the numbers in the argument
    ⍝ and then
    ÷+/inverses     ⍝ we sum those inverses and return them.
    ⍝ Of course this will give an error if 0 is in the input argument.
}

The comments do *not* affect the behaviour of the function:

In [25]:
HarmonicMean times

0.895522

### Local Variables

Notice that our `HarmonicMean` function makes use of an intermediate variable, `inverses`. Let us check its value:

In [26]:
inverses

VALUE ERROR: Undefined name: inverses
      inverses
      ∧


We got a `VALUE ERROR` because `inverses` isn't defined. It is a ***local variable***, that is, a variable that lives within the dfn only while the dfn is being executed. As soon as we exit the dfn the variable stops existing.

The notion of *local* variable is opposed to the notion of ***global variable***, which is a variable that lives in the session and thus can be accessed from anywhere. Useful global variables are functions themselves, because defining them globally means they can be used from within other functions.

As an example, we already defined the functions `Average` and `HarmonicMean` in the session. Let us now define a dfn named `AMHM` that checks empirically a mathematical theorem: that the arithmetic mean is always larger than or equal to the harmonic mean of a set of numbers:

In [27]:
]dinput
AMHM ← {
    am ← Average ⍵
    hm ← HarmonicMean ⍵
    am ≥ hm
}

In [28]:
AMHM ⍳6

1

In [29]:
AMHM times

1

Notice how the definition of `AMHM` uses both the `Average` and `HarmonicMean` dfns without defining them inside `AMHM`. This works because they were previously defined in the session.

For larger applications, proper source code management is needed and you should make sure the functions `Average` and `HarmonicMean` have been fixed when you use them inside `AMHM`, but for illustrative purposes this is absolutely fine.

### Default Left Argument

It was mentioned above that the values of `⍺` and `⍵`, the variables that represent the arguments to a dfn, cannot be assigned to. The only exception to this is when specifying a default left argument. This is relevant because a dyadic dfn can always be used monadically, as from the syntactic point of view its left argument `⍺` is always optional. If the left argument is not present it is possible to assign a default value to `⍺` by means of a normal assignment. If `⍺` is given a value because the dfn was called dyadically, such assignment is skipped.

Consider a function which calculates the $N^\text{th}$ root of a number, but which is normally used to calculate square roots ($N = 2$). You can specify that the default value of the left argument (when omitted) is 2, as follows:

In [30]:
]dinput
Root ← {
    ⍺ ← 2
    ⍵*÷⍺
}

If we don't specify the left argument of `Root`, it computes the square root. `Root` is thus said to be an ambivalent function, because it can be used both monadically and dyadically (cf. [this subsection](#Ambivalent-Functions)).

In [31]:
Root 625

25

But if we specify `⍺`, then the `⍺ ← 2` assignment is skipped:

In [32]:
4 Root 625

5

Because the assignment with `⍺←` is skipped entirely if `⍺` was provided, you should be careful with any side effects the expression to the right of `⍺←` might produce. We illustrate this with the following (silly) example:

In [33]:
]dinput
Silly ← {
    a ← 1        ⍝ This assignment always happens
    ⍺ ← a ← 2    ⍝ Not executed if ⍺ already has a value
    a            ⍝ Return a
}

In [34]:
Silly 0

2

Because we didn't provide a left argument, the `⍺ ← a ← 2` line is executed and `a` becomes 2.

On the other hand, if we provide a left argument the `⍺ ← a ← 2` line is skipped and `a` remains 1:

In [35]:
0 Silly 0

1

As for `⍵`, attempting to assign to `⍵` makes no sense: a dfn is always called monadically *or* dyadically, so the right argument is *always* present. Here's a function that computes the square root of `⍵`, except that first it tries to assign 10 to `⍵`:

In [36]:
]dinput
RootOf10 ← {
    ⍵ ← 10
    ⍵*0.5
}

Simply typing the name of the function shows its code:

In [37]:
RootOf10

{ 
 ⍵←10 
 ⍵*0.5
 }

And calling it monadically raises an error:

In [38]:
RootOf10 5

SYNTAX ERROR
RootOf10[1] ⍵←10
             ∧


### Returning the Result

We mentioned above that a dfn executes its statements until the first statement that does not assign its value. Here is a curious dfn with 4 statements:

In [39]:
]dinput
Count ← {
    1
    2
    3
    10÷0
}

Notice that all four statements are simple. If we run `Count`, what will the result be?

In [40]:
Count 73

1

The result we get is 1 because the first statement evaluates to 1 (obviously) and then we do nothing with it, so that is what the dfn returns. It doesn't matter what we wrote afterwards and it doesn't even matter that the very last statement would give a `DOMAIN ERROR`.

These superfluous statements should be avoided, as they will sooner or later cause unnecessary confusion.

As a basic debugging tool, it is possible to modify statements to display intermediate results:

In [41]:
]dinput
Count ← {
    ⎕←1
    ⎕←2
    ⎕←3
    10÷0
}

In [42]:
Count 73

1
2
3
DOMAIN ERROR: Divide by zero
Count[4] 10÷0
           ∧


Be careful: by using `⎕←` to display intermediate results, suddenly we are doing _something_ with the superfluous statements and they are all being executed (we even reached the error statement).

And even if we remove the statement that gives an error, the function will still return something other than the original 1:

In [43]:
]dinput
Count ← {
    ⎕←1
    ⎕←2
    3
}

In [44]:
Count 73

1
2
3

Now the function returned 3 instead of 1! So always be careful with which statement is actually giving the final result and avoid any extraneous statements.

## Exercises on Dfns

You are ready to solve simple problems. We **strongly recommend** that you try to solve all the following exercises before you continue further in this chapter.

**Exercise 1**:

Write a dyadic function `Extract` which returns the first `⍺` items of any given vector `⍵`.

In [45]:
Extract ← {}
3 Extract 45 86 31 20 75 62 18  ⍝ should give 45 86 31
6 Extract 'can you do it?'      ⍝ should give 'can yo'

**Exercise 2**:

Write a dyadic function which ignores the first `⍺` items of any given vector `⍵` and only returns the remainder:

In [46]:
Ignore ← {}
3 Ignore 45 86 31 20 75 62 18   ⍝ should give 20 75 62 18
6 Ignore 'can you do it?'       ⍝ should give 'u do it?'

**Exercise 3**:

Write a monadic function which returns the items of a vector in reverse order:

In [47]:
Reverse ← {}
Reverse 'snoitalutargnoc'       ⍝ should give 'congratulations'
Reverse '!ti did uoY'           ⍝ should give 'You did it!'

**Exercise 4**:

Write a monadic function which appends row and column totals to a numeric matrix.

For example, if `mat` is the matrix

In [48]:
⊢mat ← 3 4⍴75 14 86 20 31 16 40 51 22 64 31 28

75 14 86 20
31 16 40 51
22 64 31 28

Then `Totalise mat` should give

In [49]:
⊢totMat ← 4 5⍴75 14 86 20 195 31 16 40 51 138 22 64 31 28 145 128 94 157 99 478

75 14 86 20 195
 31 16 40 51 138
 22 64 31 28 145
128 94 157 99 478

Notice that `mat` occupies the upper left corner of `totMat`:

In [50]:
totMat ∊ mat

1 1 1 1 0
1 1 1 1 0
1 1 1 1 0
0 0 0 0 0

**Exercise 5**:

Write a monadic function which returns the lengths of the words contained in a text vector:

In [51]:
Lengths ← {}
Lengths 'This seems to be a good solution'    ⍝ should give 4 5 2 2 1 4 8

**Exercise 6**:

Write a dyadic function which produces the series of integer values between the limits given by its two arguments:

In [52]:
To ← {}
17 To 29    ⍝ should give 17 18 19 20 21 22 23 24 25 26 27 28 29

**Exercise 7**:

Develop a monadic function which puts a frame around a text matrix. For the first version, just concatenate minus signs above and under the matrix, and vertical bars down both sides. Then, update the function to replace the four corners by four `+` signs. For example, with

In [53]:
⊢towns ← 6 10⍴'Canberra  Paris     WashingtonMoscow    Martigues Mexico    '

Canberra 
Paris 
Washington
Moscow 
Martigues 
Mexico

we want to have `Frame towns` return

```
+----------+
|Canberra  |
|Paris     |
|Washington|
|Moscow    |
|Martigues |
|Mexico    |
+----------+
```

Finally, you can improve the appearance of the result by changing the function to use line-drawing symbols. You enter line-drawing symbols by using `⎕UCS`: the horizontal and vertical lines are

In [54]:
⎕UCS 9472 9474

─│

and the four corners are

In [55]:
⎕UCS 9484 9488 9492 9496

┌┐└┘

The final result should look like

In [56]:
⍝ ┌──────────┐
⍝ │Canberra  │
⍝ │Paris     │
⍝ │Washington│
⍝ │Moscow    │
⍝ │Martigues │
⍝ │Mexico    │
⍝ └──────────┘

**Exercise 8**:

It is very likely that the function you wrote for the previous exercise works on matrices but not on vectors. Can you make it work on both?

For example, `Frame 'We are not out of the wood'` should give

In [57]:
⍝ ┌──────────────────────────┐
⍝ │We are not out of the wood│
⍝ └──────────────────────────┘

**Exercise 9**:

Write a function which replaces a given letter by another one in a text vector. The letter to replace is given first; the replacing letter is given second, like this:

In [58]:
Switch1 ← {}
'tc' Switch1 'A bird in the hand is worth two in the bush'
⍝ A bird in che hand is worch cwo in che bush

**Exercise 10**:

Modify the previous function so that it commutes the two letters:

In [59]:
Switch2 ← {}
'ei' Switch2 'A bird in the hand is worth two in the bush'
⍝ A berd en thi hand es worth two en thi bush

## Dfns in Depth

In this section we will cover a couple of more advanced subtleties about dfns, and then we will move on to learn about tradfns.

### Guards

Previously we said that control structures cannot be used in dfns. However, it is possible to have a dfn conditionally calculate a result, by using a ***Guard***.

A *guard* is any expression which generates a one-item Boolean result, followed by a colon.

The expression placed to the right of a *guard* is executed only if the *guard* is true. In a dfn, this looks like `guard: expr` and is similar to a

```c
if (guard) {
    return expr;
}
```

of some traditional programming languages.

For example, this function will give a result equal to `'Positive'`, `'Zero'` or `'Negative'` if the argument `⍵` is respectively greater than, equal to, or smaller than zero:

In [60]:
]dinput
Sign ← {
    ⍵>0: 'Positive'
    ⍵=0: 'Zero'
    'Negative'
}

In [61]:
Sign 3

Positive

In [62]:
Sign ¯3.6

Negative

In [63]:
Sign 0

Zero

### Shy Result

Dfns can be written in such a way that they return a ***Shy result***. A *shy result* is a result which is returned, but not displayed by default.

Consider a function which deletes a file from disk and returns a result equal to 1 (file deleted) or 0 (file not found). Usually, one doesn't care if the file did not exist, so the result is not needed. But sometimes it may be important to check whether the file really existed and has been removed. So, sometimes a result is useless and sometimes it is useful... this is the reason why shy results have been invented.

In a dfn, a shy result happens when the last expression that is evaluated is assigned to a (local) name, as opposed to just leaving the result of the expression unassigned. For this to happen, one has to be careful to leave the closing curly brace `}` next to that final statement, instead of having `}` alone in a new line.

Here is the function above, written without guards and with a shy result:

In [64]:
Sign ← { s ← (3 8⍴'NegativeZero    Positive')[2+×⍵;] }

In [65]:
Sign 3

In [66]:
Sign ¯3.6

In [67]:
⎕← Sign 0

Zero

Notice what happens if we were to format `Sign` as we have formatted previous dfns:

In [68]:
]dinput
Sign ← {
    s ← (3 8⍴'NegativeZero    Positive')[2+×⍵;]
}

In [69]:
Sign 3

In [70]:
Sign ¯3.6

In [71]:
⎕← Sign 0

VALUE ERROR: No result was provided when the context expected one
      ⎕←Sign 0
        ∧


The `VALUE ERROR` we get above is a very subtle error. Because the `s ← ...` statement is an assignment, when executing the `Sign` dfn the interpreter goes on to execute the expression on the next line, but the next line has *no* expression and so the interpreter raises a `VALUE ERROR`. If we want to have a shy result on a multi-line dfn, we *must* have the final curly brace on the same line as the final statement.

### Lexical Scoping

***Lexical scoping*** (also referred to as ***static scoping***) is the mechanism that turns local and global variables into relative notions that depend on the context in which dfns were defined: dfns usually have access to global variables, but the variables that are "global" depend on where the dfn was written.

As a purely illustrative example, consider the function defined below:

In [72]:
]dinput
MultiplyBy10← {
    v ← 10            ⍝ define some variable
    TimesV ← {v×⍵}    ⍝ multiply something with v
    TimesV ⍵
}

In [73]:
MultiplyBy10 5

50

In [74]:
MultiplyBy10 10

100

Notice how the `MultiplyBy10` function takes your input and gives it to the `TimesV` function, which is defined as a function that "*takes its input (`⍵`) and multiplies it with `v`*". But what is `v`? We do not give a value to `v` inside `TimesV`, so when APL encounters the expression `v×⍵` it looks at its surroundings for the meaning of `v`. Because `v` was defined in the enclosing dfn as `v ← 10`, that is the value that is used.

Consider now a similar example, but with more occurrences of the variable `v`:

In [75]:
v ← 100          ⍝ (1)

In [76]:
]dinput
MultiplyBy10 ← {
    GiveMeV ← {
        v
    }
    ⎕← v         ⍝ (3)
    ⎕← GiveMeV 1 ⍝ (4)
    v ← 10       ⍝ (5)
    ⎕← v         ⍝ (6)
    ⎕← GiveMeV 1 ⍝ (7)
    v ← 10×⍵     ⍝ (8)
    v
}

In [77]:
⎕← v             ⍝ (2)
MultiplyBy10 3
⎕← v             ⍝ (9)

100

100
100
10
10
30

100

Let us go over the assignments and the outputs of the code above:

 - we start by defining the variable `v` in our session and we set it to 100 (1);
 - we then define a function named `MultiplyBy10` which happens to contain another dfn inside it;
 - then we print the value of the session variable `v` and we see its value is 100 (2);
 - then we call the dfn `MultiplyBy10` with argument `3` and
   - we define a new dfn named `GiveMeV`;
   - we print the value of `v` (3). `MultiplyBy10` doesn't know what `v` is and so it looks for it in the session and finds a `v` whose value is 100, because of (1);
   - we then call the function `GiveMeV` which simply returns `v` and we print it (4). `GiveMeV` doesn't know what `v` is, so it asks `MultiplyBy10`, which in turns asks the session, which knows of a `v` whose value is 100, because of (1);
   - we then define `v` to be 10 inside of `MultiplyBy10` (5), making `MultiplyBy10` aware of a variable `v`;
   - then we print `v` inside `MultiplyBy10` (6), which is 10 because we just defined it as such;
   - then we call `GiveMeV` again and we print its result (7). `GiveMeV` doesn't know what `v` is, so it asks `MultiplyBy10`, that *now knows* what `v` is: it is 10 because of (5);
   - and we finish executing the `MultiplyBy10` dfn by assigning 30 to `v` (8), which we then return from the dfn;
 - we leave the `MultiplyBy10` dfn call and 30 gets printed because that was the result of the dfn call;
 - finally we print `v` once more, and the session knows `v` is 100, so that is what we print (9).
 
That might look confusing, but I assure you it makes a lot of sense. Just go through the code calmly and make sure you understand what each part does separately. Then, simulate the execution of the code with some pen and paper and write down what you think should get printed at each step. Then read the explanation above and compare it to what you thought was supposed to happen. You will get used to lexical scoping in no time.

Lexical scoping can reveal itself to be extremely useful in languages where functions can return other functions, which is *not* the case with APL. Even so, lexical scoping will prove to be helpful later down the road: imagine a large(r) function inside which we define a small utility dfn to use with an operator, but we want the dfn to make use of things we have already computed inside the outer function. Lexical scoping kicks in at that point, allowing the inner dfn to access everything the outer function already computed.

This concludes the more advanced topics on dfns. If you are used to programming in other programming languages that follow paradigms other than the array-oriented one, dfns may look very limited in their lack of conventional control structures and looping structures. As it turns out, in writing good array-oriented programs you can usually do away without these. What is more, the power of many of the built-in operators you will learn about in [a future chapter](./Operators.ipynb) will cover that gap in a very suitable way.

## Tradfns

***Tradfns***, which were previously referred to as *procedural functions*, are mainly used for complex calculations involving many variables, interactions with a user, file input/output of data, etc. They look much like functions or programs in more traditional programming languages.

Tradfn is short for "*traditional function*", because in the beginning APL did not have support for dfns. Hence, when dfns were a novelty, tradfns were the *traditional* functions that had been around for a while.

### A First Example

Tradfns are composed of a *Header* and one or more *statements* (function lines), so invoking a text editor to enter these lines of text will make your life easier.

As an example, let us see how we could define the `HarmonicMean` from before as a tradfn. Let us call it `TradHarmonicMean`, for *traditional harmonic mean*.

Let us open the editor with `)ED TradHarmonicMean` and outright define our function:

In [78]:
∇ mean ← TradHarmonicMean argVector
    inverses ← ÷argVector
    mean ← ÷+/inverses
∇

Now I will break it down for you:

The function is delimited by a pair of `∇` symbols. This special symbol is named "***Del***" in English, or "***Carrot***" (because of its shape) in some French-speaking countries. You can type a *Del* with <kbd>APL</kbd>+<kbd>g</kbd>. In Jupyter notebooks and in the session, those are mandatory. If you define the tradfn in the text editor, you can omit them (cf. <!--figure-->the figure below<!--Editor_TradHarmonicMean-->);
 
The first line of the tradfn is the header and tells APL that
 - the tradfn is going to be called `TradHarmonicMean`,
 - it expects a right argument which will be named `argVector` and
 - it will return the value we store in the variable named `mean`.
   
The subsequent lines have the statements of the function itself and, in particular, the final line in which we perform the `mean ← ...` assignment is where we establish the result that will be returned by the tradfn.
 
![A tradfn defined in the editor and without delimiting Del characters](res/Editor_TradHarmonicMean.png)

The function is now available for use:

In [79]:
TradHarmonicMean 2 6

1.5

### The Default Isn't Local

Just like when we defined `HarmonicMean` as our first dfn, our tradfn uses a temporary variable called `inverses`. Let us check its value:

In [80]:
inverses

0.5 0.166667

When we did the same check in [the subsection "Local Variables"](#Local-Variables) we couldn't access the value of `inverses` because it was not defined in the session, it was a variable that was local to the dfn. Clearly, this works differently in tradfns.

If you were paying attention, you will have noticed that in <!--figure-->the figure above<!--Editor_TradHarmonicMean--> there's two colours being used (the actual colours might differ between the Windows IDE and RIDE, and you might also have changed your colour scheme to something else):

 - black for the names `TradHarmonicMean` and `inverses`;
 - grey for the names `argVector` and `mean`.
 
So what do the colours mean?

The names in black are the variables that are global, variables that remain in the workspace after they have been assigned values during execution of the function. These names can refer to existing variables - intentionally or not - and may produce undesirable side effects.

The names in grey are temporary variable names used during function execution. Once execution is complete, these temporary variables are destroyed. Right before that, the tradfn returns the *value* of the return variable - `mean` in our example - and then discards its *name*.

That means neither `argVector` nor `mean` should be available after execution of the tradfn ends:

In [81]:
mean

VALUE ERROR: Undefined name: mean
      mean
      ∧


In [82]:
argVector

VALUE ERROR: Undefined name: argVector
      argVector
      ∧


Because `inverses` is coloured in black, we can see it can interfere with the variables in our workspace:

In [83]:
inverses ← 'The inverses are calculated by use of monadic ÷'
TradHarmonicMean 1 2 3 4

0.48

And suddenly `inverses` is no longer what you defined:

In [84]:
inverses

1 0.5 0.333333 0.25

All the variables created during execution of the tradfn that are not referenced in the header are considered to be global variables. To avoid any unpredictable side effects, it is recommended that you declare as local all the variables used by a function. This is done by specifying their names in the header each prefixed by a ***semi-colon***, as shown in this new definition of `TradHarmonicMean`:

In [85]:
∇ mean ← TradHarmonicMean argVector; inverses
    inverses ← ÷argVector
    mean ← ÷+/inverses
∇

In [86]:
inverses ← 'The inverses are calculated by use of monadic ÷'
TradHarmonicMean 1 2 3 4

0.48

In [87]:
inverses

The inverses are calculated by use of monadic ÷

As we can see, `inverses` is now treated as a local variable.

<!--begin rules-->
***Rules***:

 >  - All the names referenced in the header of a function (including its result and arguments) are ***local*** to the function. They will exist only during the execution of the function.
 >
 >  - Operations made on local variables do not affect ***global*** variables having the same names.
 >
 >  - *Global* and *Local* are relative notions: when a tradfn calls another sub function, variables local to the calling function are global for the called function. This will be further explored in [a bit](#Dynamic-Scoping).
 >
 >  - All the variables used in a tradfn should preferably be declared local, unless you specifically intend otherwise.
<!--end-->

### Defining Sub-Functions

As we have seen, dfns can be defined inside other dfns. Similarly, dfns can be defined inside tradfns and used right away. The precautions mentioned above, that need to be taken with respect to *global* versus *local* variables, still apply. In short, do not forget to localise the name of a temporary dfn unless you really mean for it to become global.

### A Second Example

You may remember from previous chapters our two matrices `forecast` and `actual`, representing sales of 4 products over 6 months:

In [88]:
⎕RL ← 73
⊢forecast ← 10×?4 6⍴55

90 160 420 500 20 30
110 450 170 370 290 360
340 190 320 120 510 370
150 460 240 520 490 280

In [89]:
⎕RL ← 73
⊢actual ← forecast + ¯10+?4 6⍴20

89 166 420 508 12 23
111 453 177 365 284 352
349 192 329 115 515 374
160 467 234 519 485 283

It would be nice to interlace the columns of those two matrices to make it easier to compare forecast and actual sales for the same month. Furthermore, because this might be useful for other pairs of matrices, let's create a general function to do the job; let's call it `Interlace`.

The result we would like to obtain is schematised in the table below, with `f` being shorthand for `forecast` and `a` being shorthand for `actual`:

| `f[;1]` | `a[;1]` | `f[;2]` | `a[;2]` | `f[;3]` | `a[;3]` | `f[;4]` | `a[;4]` | `f[;5]` | `a[;5]` | `f[;6]` | `a[;6]` |
| :-: | :-: | :-: | :-: | :-: | :-: | :-: | :-: | :-: | :-: | :-: | :-: |
| 90 | 89 | 160 | 166 | 420 | 420 | 500 | 508 | 20 | 12 | 30 | 23 |
| 110 | 111 | 450 | 453 | 170 | 177 | 370 | 365 | 290 | 284 | 360 | 352 |
| 340 | 349 | 190 | 192 | 320 | 329 | 120 | 115 | 510 | 515 | 370 | 374 |
| 150 | 160 | 460 | 467 | 240 | 234 | 520 | 519 | 490 | 485 | 280 | 283 |

The first thing you must decide is how this function will be used:

 - will you pass both matrices on right: `Interlace forecast actual`
 - or one on the left and one on the right: `forecast Interlace actual`
 
Both solutions are valid; it is only a question of personal taste and ease of use. Our first tradfn was monadic so let us make this one dyadic.

Having decided on the calling syntax, we just have to know how the header of a dyadic tradfn looks like. If we call `r` to the result and `x` and `y` to the left and right arguments, respectively, then the tradfn header is

In [90]:
∇ r ← x Interlace y
∇

which may then be updated to include local variables.

The names `x`, `y` and `r` do not matter to the APL system. Replacing `x` and `y` with `a` and `b` or `left` and `right` would be perfectly valid and obviously easier to recall than if we replaced them with `potatoes` and `ocarina`. However, you should pick names which help you remember what the function is doing. For example, in a general-purpose function like this, you should probably avoid using too specific names like `forecast` and `actual`: that would imply that this function only works on arrays containing forecast and actual data. Such names might also confuse the distinction between local and global names.

Now that the header is set, how shall we interlace our two matrices? Have a go at it, if you fancy.

We suggest the following steps:

 - calculate the size of the result `r`. It will be a matrix with as many rows as `x` and `y`, but twice as many columns;
 - create `r` filled with zeroes;
 - calculate the indices of its even columns;
 - fill the even columns with `y`;
 - calculate the indices of its odd columns and fill them with `x`.
 
The final function could be written as follows. Do not forget to localise your variables.

In [91]:
∇ r ← x Interlace y; size; even
    size ← 1 2×⍴x
    r ← size⍴0
    even ← 2×⍳(⍴x)[2]
    r[;even] ← y
    r[;even-1] ← x
∇

We can now apply the function to any pair of variables, provided they have the same size:

In [92]:
forecast Interlace actual

90 89 160 166 420 420 500 508 20 12 30 23
110 111 450 453 170 177 370 365 290 284 360 352
340 349 190 192 320 329 120 115 510 515 370 374
150 160 460 467 240 234 520 519 490 485 280 283

In [93]:
(2 3⍴⍳6) Interlace (2 3⍴5 7 0 2 8 9)

1 5 2 7 3 0
4 2 5 8 6 9

**Another possible syntax**

If you had decided instead to make the function monadic, it could look very similar to the one above. We just changed the header a bit and added a new statement at the beginning:

In [94]:
∇ r ← Interlace couple; x; y; size; even
    (x y) ← couple
    size ← 1 2×⍴x
    r ← size⍴0
    even ← 2×⍳(⍴x)[2]
    r[;even] ← y
    r[;even-1] ← x
∇

In [95]:
Interlace forecast actual

90 89 160 166 420 420 500 508 20 12 30 23
110 111 450 453 170 177 370 365 290 284 360 352
340 349 190 192 320 329 120 115 510 515 370 374
150 160 460 467 240 234 520 519 490 485 280 283

### Dynamic Scoping

***Dynamic scoping*** (as opposed to *static* scoping, as seen in [here](#Lexical-Scoping)) is the mechanism that turns local and global variables into relative notions that depend on the chain of function calls that are currently being interpreted: when a tradfn calls another function, variables local to the calling function are global for the called function.

To explore what this means, consider the following tradfns. Do not worry about their arguments and return values as they are not meaningful. What matters are the assignments to `a` and `b` and the prints.

In [96]:
∇ r ← PrintAB y
    ⎕← a
    ⎕← b
    r ← 0
∇

In [97]:
∇ r ← SetAThenPrint y; a
    a ← 1
    r ← PrintAB y
∇

In [98]:
∇ r ← SetBThenPrint y; b
    b ← 2
    r ← PrintAB y
∇

and let us define *global* variables `a` and `b` with character values:

In [99]:
a ← 'a'
b ← 'b'

Now let us call `SetAThenPrint`. What do you expect to be printed?

In [100]:
SetAThenPrint 0

1
b
0

Did it match your expectations? Can you now guess what will happen if we execute `SetBThenPrint`?

In [101]:
SetBThenPrint 0

a
2
0

Notice how `PrintAB` was defined as a standalone tradfn which prints two variables. Because `a` and `b` are never defined inside `PrintAB`, surely `PrintAB` will be printing global values. Then, depending on what function called `PrintAB`, either `a` or `b` are taken from the context of the enclosing *calling* function.

Let us try to define the same functions but with dfns:

In [102]:
]dinput
PrintCD ← {
    ⎕← c
    ⎕← d
    0
}

In [103]:
]dinput
SetCThenPrint ← {
    c ← 3
    PrintCD ⍵
}

In [104]:
]dinput
SetDThenPrint ← {
    d ← 4
    PrintCD ⍵
}

In [105]:
c ← 'c'
d ← 'd'

What will happen if we call `SetCThenPrint`?

In [106]:
SetCThenPrint 0

c
d
0

Even though `SetCThenPrint` modified the value of `c` to be 3, the `PrintCD` function was defined in the workspace, and hence its lexical scoping tells it to look for `c` among its own variables, and among the global variables in the workspace. `PrintCD` does not have access to the local variables of the function that called it.

That is why, if we call `SetDThenPrint`, we get the exact same output:

In [107]:
SetDThenPrint 0

c
d
0

Perhaps the difference between dynamic and lexical scoping becomes even more clear if we consider yet another family of tradfns:

In [108]:
∇ r ← PrintE y
    ⎕← e
    r ← 0
∇

Notice that if you call `PrintE` right now it will cause an error, because there is no variable `e` defined anywhere:

In [109]:
e

VALUE ERROR: Undefined name: e
      e
      ∧


In [110]:
PrintE 0

VALUE ERROR: Undefined name: e
PrintE[1] ⎕←e
            ∧


Now let us define a tradfn that makes use of a *local* variable `e`:

In [111]:
∇ r ← SetLocalE y; e
    e ← 5
    r ← PrintE y
∇

If we call `SetLocalE`, `e` will be defined inside `SetLocalE` and then, when `PrintE` gets called, the dynamic scoping will make `e` visible as a global variable:

In [112]:
SetLocalE 0

5
0

After `SetLocalE` is executed, our workspace still has no variable `e`:

In [113]:
e

VALUE ERROR: Undefined name: e
      e
      ∧


We now summarise the scoping rules used when a function `F` calls another function `G`.

<!--begin scoping rules style=tip-->
***Scoping rules***:

 >  - If `F` is a tradfn, then all of `F`'s local variables will be visible to `G`. 
 >
 >  - If `F` is a dfn and you want its local variables to be visible to `G`, then either define `G` as a tradfn or define `G` as a dfn *inside* `F`.
<!--end-->

## Flow Control in Tradfns

### Overview

Apart from extremely simple calculations, most programs rely on certain statements being executed only if a given condition is satisfied (conditional execution) or on a set of statements being executed again and again, until a given limit is reached (looping). The APL language offers a special set of syntactic elements to control the flow of statements inside tradfns.

In the very first versions of APL, the only way to implement conditional execution and looping was to use the symbol `→` (the *branch arrow*, typed with <kbd>APL</kbd>+<kbd>]</kbd>). This was used to jump from one statement to another, skipping over other statements (conditional execution) or jumping back to repeat a set of statements again (looping). The branch arrow is equivalent to the GOTO statement in other languages. Contemporary versions of APL include a special set of keywords which offer a much more flexible, easy to use and easy to read way to control the flow of execution. They are also very similar to those used in most other languages. These are known as ***Control structures***.

We shall begin by using control structures and then introduce you later to the old way of programming, only because you may come across it in some existing programs, and because it sometimes offers shorter or more convenient ways of doing things.

Control structures are blocks of statements which begin and end with special keywords. These keywords are all **prefixed** with a ***colon***, like `:If` or `:Repeat`. This prevents the names of your variables to clash with these keywords.

The keywords can be typed in lower or upper case, but Dyalog APL will always store and display them using a fixed spelling convention commonly known as CamelCase or, more specifically, PascalCase. Following this convention, a keyword has an upper case first letter and the following letters in lower case. Composite keywords like "`EndIf`" and "`GoTo`" capitalise the first letter of each word.

Opening keywords are used to begin the conditional execution or repeated execution of a block of statements. Usually (but not always), the block is ended by a corresponding keyword that starts with `:End`.

The sets of opening/closing keywords are shown below:

| Opening | Closing |
| :- | :- |
| `:If` | `:EndIf` |
| `:For` | `:EndFor` |
| `:Select` | `:EndSelect` |
| `:Repeat` | `:Until` |
| | `:EndRepeat` |
| `:While` | `:EndWhile` |
| | `:Until` |
| `:Trap` | `:EndTrap` |
| `:With` | `:EndWith` |
| `:Hold` | `:EndHold` |

The primary keywords shown above can be complemented by additional keywords which qualify more precisely what is to be done:

 - `:Else`
 - `:ElseIf`
 - `:AndIf`
 - `:OrIf`
 - `:Case`
 - `:CaseList`
 - `:Until`
 
And finally, some keywords may be used to conditionally alter the flow of execution within a control structure.

The following keywords will not be studied in this chapter:

 - `:Trap... :EndTrap` concerns event processing and will be seen in [the chapter "Event Handling"](./Event-Handling.ipynb);
 - `:With... :EndWith` concerns the GUI interface and namespaces and will be covered in [the chapter on GUIs](./Graphical-User-Interface.ipynb) and in [the chapter on namespaces](./Namespaces.ipynb);
 - `:Hold... :EndHold` concerns *multi threading*.

### Conditional Execution

#### Simple Conditions (`:If`/`:EndIf`)

The clauses `:If` and `:EndIf` delimit a block of statements (Block **1** in <!--figure-->the diagram below<!--Control_Flow_If_EndIf-->), which will be executed only if the condition specified by the `:If` clause is satisfied.

![A diagram with the usage of `:If... :EndIf`](res/Control_Flow_If_EndIf.png)

Here is the explanation of what happens inside the function of the diagram <!--figure-->above<!--Control_Flow_If_EndIf-->:

 - If present, Block **0** will always be executed, as will Block **2**;
 - *Condition* is any expression whose result is a Boolean **scalar** or **one item array**; for example, `code∊list`, `price>100` or `values∧.=0`;
 - Block **1** will be executed if *Condition* is satisfied.
 
<!--begin example-->
***Example***:

 > Our keyboard has been damaged: we can no longer use the *Absolute value* key. Perhaps a tradfn could replace it? Try writing a tradfn using an `:If` clause.
<!--end-->

Here is a possible solution:

In [114]:
∇ y ← AbsVal y
    :If y < 0
        y ← -y
    :EndIf
∇

In [115]:
AbsVal 1

1

In [116]:
AbsVal ¯3.4

3.4

If the argument is positive (or zero), the function does nothing and just returns the argument it received. If the argument is negative, it returns the corresponding positive value.

#### Alternative Processing (`:If`/`:Else`/`:EndIf`)

In the previous example if *Condition* is satisfied, Block **1** is executed; otherwise nothing is done. But sometimes we would like to execute one set of statements (Block **1**) if *Condition* is satisfied or an alternative one (Block **2**) if it is not.

For this, we use the additional keyword `:Else` as shown in <!--figure-->the diagram below<!--Control_Flow_If_Else_EndIf-->:

![A diagram with the usage of `:If... :Else... :EndIf`](res/Control_Flow_If_Else_EndIf.png)

Blocks **0** and **3** will always be executed, if present. Block **1** will be executed if *Condition* **is** satisfied and Block **2** will be executed if *Condition* is **not** satisfied. Notice this means that exactly *one* of the two blocks gets executed. Never both, and never none.

<!--begin example-->
***Example***:

 > Let us try to find the real roots of the quadratic equation $ax^2 + bx + c = 0$, given the values of $a$, $b$ and $c$. If the quantity $\Delta = b^2 - 4ac$ is negative, it is known that the two solutions of this equation are complex numbers. $\Delta$ is commonly referred to as the [discriminant](https://en.wikipedia.org/wiki/Quadratic_equation#Discriminant) of the equation. 
<!--end-->

We can write a tradfn that computes the two roots by means of the [quadratic formula](https://en.wikipedia.org/wiki/Quadratic_equation#Quadratic_formula_and_its_derivation), except that it issues an error message if there are no real roots. We will use an `:If... :Else... :EndIf` clause for this:

In [117]:
∇ r ← QuadRoot abc; a; b; c; delta
    (a b c) ← abc
    delta ← (b*2)-4×a×c
    :If delta≥0
        r ← (-b)+1 ¯1×delta*0.5
        r ← r÷2×a
    :Else
        r ← 'No roots'
    :EndIf
∇

In [118]:
QuadRoot ¯2 7 15

¯1.5 5

In [119]:
QuadRoot 1 1 1

No roots

#### Composite Conditions (`:OrIf`/`:AndIf`)

Multiple conditions can be combined using the Boolean functions "OR" and "AND".

Consider the diagram <!--figure-->below<!--Control_Flow_OrIf-->:

![A diagram with the usage of `:OrIf`](res/Control_Flow_OrIf.png)

If present, Block **0** and Block **3** will always be executed. Block **1** and *Condition* **2** are only executed if *Condition* **1** is **not** satisfied. Block **2** is executed if *Condition* **1** **OR** *Condition* **2** **is** satisfied.

In many cases, the same result could be obtained by a more traditional APL approach using `∨`: `:If (Condition 1) ∨ (Condition 2)`.

However, suppose that Block **1** and/or *Condition* **2** need a lot of computing time.

 - The traditional APL solution will always evaluate both *Condition* **1** and *Condition* **2**, combine the results, and decide what to do.
 - With the "`:OrIf`" technique, if *Condition* **1** is satisfied, Block **2** will be immediately executed, and neither Block **1** nor *Condition* **2** will be evaluated. This may sometimes save a lot of processing time.
 
Using the `:OrIf` clause will thus enable what is usually referred to as *short-circuiting* in other programming languages.
 
Note that the optional Block **1** may be useful to prepare the variables to be referenced in *Condition* **2**, but can also be omitted.

We have a similar structure with the `:AndIf` clause, as seen in <!--figure-->this next diagram<!--Control_Flow_AndIf-->:

![A diagram with the usage of `:AndIf`](res/Control_Flow_AndIf.png)

Again, Block **0** and Block **3** will always be executed. Block **1** is optional and will only be executed if *Condition* **1** **is** satisfied. Likewise, *Condition* **2** will be executed only if *Condition* **1** is satisfied. Finally, Block **2** is executed if both *Condition* **1** **AND** *Condition* **2** are satisfied.

<!--begin rule-->
***Rule***:

 > For `:OrIf` and `:AndIf`, Block **1** and *Condition* **2** are only executed if the result of *Condition* **1** is **not** enough to determine the result of the Boolean operation combining conditions **1** and **2**.
<!--end-->

In many cases, the same result could be obtained by a more traditional APL approach using `∧`: `:If (Condition 1) ∧ (Condition 2)`.

However, it may be that *Condition* **2** cannot be evaluated if *Condition* **1** is not satisfied. For example, we want to execute Block **2** if the variable `var` exists and is smaller than the argument `arg`. It is obvious that `var<arg` cannot be evaluated if the variable `var` does not even exist. The two conditions must be evaluated separately:

In [120]:
∇ r ← CheckVar arg
    r ← 0
    :If 2=⎕NC'var'
        ⎕← 'var exists'
    :AndIf var<arg
        ⎕← 'var is smaller than arg'
        r ← 1
    :EndIf
∇

Notice that `var` doesn't exist as a variable in our workspace:

In [121]:
var

VALUE ERROR: Undefined name: var
      var
      ∧


Hence `CheckVar` should do nothing and return 0:

In [122]:
CheckVar 1000

0

If *Condition* **1** is not satisfied neither Block **1** nor *Condition* **2** will be executed. This may also save some computing time in more complex tradfns.

Now we set `var` to some value and we can then check it:

In [123]:
var ← 500

In [124]:
CheckVar 1000

var exists
var is smaller than arg
1

In [125]:
CheckVar 100

var exists
0

Note that you may not combine `:OrIf` and `:AndIf` within the same control structure. The following code will generate a `SYNTAX ERROR`:

In [126]:
∇ surface ← BadSyntaxTradfn args; width; length; height
    (width length height) ← args
    :If width<20
    :AndIf length<100
    :OrIf height<5
        surface ← 0
    :Else
        surface ← width×length
    :EndIf
∇

In [127]:
BadSyntaxTradfn 5 5 5

SYNTAX ERROR
BadSyntaxTradfn[2] :If width<20
                            ∧


#### Cascading Conditions (`:ElseIf`/`:Else`)

Sometimes, if the first condition is not satisfied, perhaps a second or a third one will be. In each case, a different set of statements will be executed. This type of logic may be controlled by one or more "`:ElseIf`" clauses. And if none of these conditions are satisfied, perhaps another block of statements is to be executed; this may be controlled by a final "`:Else`", as we have seen earlier.

Depending on the problem, "`:Else`" may be present or not. If there is no "`:Else`" clause and no condition has been satisfied, nothing will be executed inside the `:If` block.

![A diagram with multiple usages of `:ElseIf` clauses](res/Control_Flow_ElseIf.png)

By now, you should be able to tell by yourself that Block **0** and Block **5** in <!--figure-->the diagram above<!--Control_Flow_ElseIf-->, if present, will always be executed.

For the conditions and the remaining blocks, a simple rule applies:

<!--begin rule-->
***Rule***:

Using <!--figure-->the diagram above<!--Control_Flow_ElseIf--> for the examples:

 > - The conditions are executed in turn, until one of them is satisfied. When one condition is satisfied, its corresponding block is executed. For example, if *Condition* **1** is not satisfied and *Condition* **2** **is** satisfied, then Block **2** **is** executed.
 >
 > - As soon as one condition is satisfied and its block is executed, all other conditions and blocks inside the `:If... :EndIf` control structure are ignored. For example, if *Condition* **2** is the first condition to be satisfied, then *Condition* **3** is **not** executed (**even** if *Condition* **3** would be satisfied), as neither are Block **3** nor Block **4**.
 >
 > - If none of the conditions is satisfied and an `:Else` clause is present, then its corresponding block is executed. For example, if conditions **1** to **3** are not satisfied, then Block **4** is satisfied.
 >
 > Hence, all conditional blocks (blocks **1** to **4**) are mutually exclusive.
<!--end-->

For example, suppose that the first condition is `var<100` and the second is `var<200`.

If `var` happens to be equal to 33, it is both smaller than 100 and 200, but only the block of statements attached to `var<100` will be executed:

In [128]:
∇ r ← VarLevels var
    :If var<100
        ⎕← 'var is < than 100'
        r ← 100
    :ElseIf var<200
        ⎕← 'var is < than 200'
        r ← 200
    :Else
        ⎕← 'var is too big'
        r ← var
    :EndIf
∇

In [129]:
VarLevels 33

var is < than 100
100

In [130]:
VarLevels 133

var is < than 200
200

In [131]:
VarLevels 350

var is too big
350

#### Alternative Solutions

Now you know how to use control structures to write conditional expressions. However, this does not mean that you always *have to* or *should* use control structures. The richness of the APL language often makes it more convenient to express condition calculations using a more mathematical approach.

For example, suppose that you need to comment on the result of a football or rugby match by displaying "*Won*", "*Draw*" or "*Lost*", depending on the scores of the two teams. Here are two solutions:

In [132]:
∇ r ← x Comment y
    :If x>y
        r ← 'Won'
    :ElseIf x=y
        r ← 'Draw'
    :Else
        r ← 'Lost'
    :EndIf
∇

In [133]:
2 Comment 1

Won

In [134]:
2 Comment 2

Draw

In [135]:
0 Comment 3

Lost

In [136]:
∇ r ← x Comment y
    which ← 2+×x-y
    r ← (3 4⍴'LostDrawWon ')[which;]
∇

In [137]:
2 Comment 1

Won

In [138]:
2 Comment 2

Draw

In [139]:
0 Comment 3

Lost

Which solution you prefer is probably a matter of taste and previous experience, both yours and of whoever is to read and maintain the programs you write. Notice that only the second solution is suitable for a dfn.

### Disparate Conditions

#### Clauses (`:Select`/`:Case`/`:CaseList`)

Sometimes it is necessary to execute completely different sets of statements, depending on the value of a specific *control expression*, hereafter called the *control value*.

To achieve this, we use "`:Select`", with additional "`:Case`" or "`:CaseList`" clauses.

The sequence begins with `:Select` followed by the *control expression*.

It is then followed by any number of blocks, each of which will be executed if the *control value* is equal to one of the values specified in the corresponding clause:

 - `:Case` for a single value.
 - `:CaseList` for a list of possible values.
 
The sequence ends with `:EndSelect`.

You can have as many `:Case` or `:CaseList` clauses as you need and in any order.

If the control value is not equal to any of the specified values, nothing is executed, unless there is one final `:Else` clause.

Like with `:If... :ElseIf... :Else`, the blocks are mutually exclusive. The `:Case` statements are examined from the top and once a match is found and the corresponding block of statements has been executed, execution will continue with the first line after the `:EndSelect` statement - even if the *control value* matches other `:Case` or `:CaseList` statements.

In <!--figure-->the diagram below<!--Control_Flow_Select-->, the *control expression* is simply `District` and the *control value* is whatever value the variable holds. Then, Block **1** is executed if `District` is equal to 50, Block **2** is executed if `District` is equal to 19 and Block **3** is executed if `District` is equal to 41, 42 or 53, but not if `District` is equal to 50. In that case, Block **1** will already have been executed and the execution won't reach this `:CaseList`. Therefore, we conclude that including 50 in this `:CaseList` is superfluous and we better remove it to avoid any confusion. Finally, Block **4** is only executed if `District` is not equal to any of the values listed above.

![A diagram with the usage of `:Select`, `:Case` and `:CaseList`](res/Control_Flow_Select.png)

<!--begin remark-->
***Remark***:

 > Values specified in `:Case` or `:CaseList` clauses can be numbers, characters or even nested arrays.
<!--end-->

Here is an example using all of those:

In [140]:
∇ arg ← CasePotpourri arg
    :Select arg
    :CaseList 'yes' 'no' 'doubt'  ⍝ 3 possible values
        ⎕← 'yes no doubt'
    :CaseList (2 7)(5 1)'Null'    ⍝ 3 different possible vectors
        ⎕← 'some vector'
    :Case 'BERLIN'                ⍝ 1 single word
        ⎕← 'Germany'
    :CaseList 'PARIS'             ⍝ 5 possible letters
        ⎕← 'A French letter'
    :Else
        ⎕← 'no match...'
    :EndSelect
∇

In [141]:
CasePotpourri 'yes'

yes no doubt
yes

In [142]:
CasePotpourri 2 7

some vector
2 7

In [143]:
CasePotpourri 'I'

A French letter
I

In [144]:
CasePotpourri 'BERLIN'

Germany
BERLIN

In [145]:
CasePotpourri 'PARIS'

no match...
PARIS

Be careful with the last 3 examples where a character vector is used, and the relevant `:Case 'BERLIN'` and `:CaseList 'PARIS'` clauses in the tradfn:

 - if the keyword is `:Case`, the control value must match the entire character vector "`'BERLIN'`";
 - if the keyword is `:CaseList`, the control value may be any one letter out of the 5 letters in "`'PARIS'`". Any subset, like "`'PAR'`" will not be recognised as matching.
 
<!--begin warning-->
***Warning***:

 > The control value must be **strictly** identical to the value(s) specified in the `:Case` clause(s), i.e. the result of using `≡` with the control value and the matching value in the clause should be 1.
<!--end-->

For example, in the preceding diagram, there is a clause `:Case 50` (scalar). If the control value is equal to `1⍴50` (a one-item vector), it is not strictly identical to the specified array (the scalar `50`) and the corresponding set of statements will **not** be executed. In fact,

In [146]:
50≡(1⍴50)

0

### Predefined Loops

#### Basic Use (`:For`/`:In`/`:EndFor`)

In many iterative calculations a set of statements is repeated over and over again, and on each iteration a new value is given to a particular variable. We will refer to this variable as the *control variable*.

If the values of the control variable can be predefined before the beginning of the loop, we recommend that you use the `:For` clause, with the following syntax: `:For controlVariable :In listOfValues`.

The keyword `:For` is followed by the name of the control variable. In the same statement, the keyword `:In` is followed by an expression returning the list of values to be assigned to the control variable on each iteration. Here is an example:

In [147]:
∇ r ← ZapMe y
    r ← 0
    :For zap :In 50 82 27 11
        ⎕← zap
        r ← zap
    :EndFor
∇

In [148]:
ZapMe 3

50
82
27
11
11

The block of statements between the `:For` and `:EndFor` will be executed 4 times: once with `zap ← 50`, then with `zap ← 82`, again with `zap ← 27` and finally with `zap ← 11`.

Generally, the block of statements makes some reference to the *control variable*, for example as part of a calculation, but this is not mandatory.

This technique has one great advantage: the number of loops is predefined, and it is impossible to accidentally program an endless loop.

#### Control of Iterations

The values assigned to the control variable can be whatever values are needed by the algorithm:

 - a list of numeric values like `66+4×⍳20`;
 - a nested vector like `(5 4)(3 0 8)(4 7)(2 5 9)`;
 - a list of letters like `'DYALOG'`;
 - a list of words like `'Madrid' 'Paris' 'Tokyo' 'Ushuaia'`.
 
It is also possible to use a *set* of control values, rather than just a single one.

For example, with `:For (code qty) :In (5 8)(2 3)(7 4)` the loop will be executed

 - first with `code ← 5` and `qty ← 8`,
 - then with `code ← 2` and `qty ← 3`
 - and finally with `code ← 7` and `qty ← 4`.
 
In most cases, this kind of iterative process is executed to completion. However, it is possible to take an early exit when some condition or other is met. This can be done using the `:Leave` clause or using an explicit branch, like will be explained later.

A special variant of `:In` named `:InEach` is explained in [the Specialist's Section](#Specialists-Section) at the end of this chapter.

<!--begin example-->
***Example***:

 > Let us try to find all the possible divisors of a given integer.
<!--end-->

A possible solution is to check the remainder of that value against all integers starting from 1, up to the number itself. If the remainder is 0, the integer can be appended to the vector of results, which has been initialised as an empty vector:

In [149]:
∇ divs ← Divisors n; rem; div
    divs ← ⍬
    :For div :In ⍳n
        rem ← div|n
        :If rem=0
            divs ← divs,div
        :EndIf
    :EndFor
∇

In [150]:
Divisors 3219

1 3 29 37 87 111 1073 3219

This example hopefully shows that it is straightforward to write a simple, predefined loops using control structures. If you are used to other programming languages that do not offer array processing features, you may even find this way of writing programs very natural.

However, it turns out that many simple, predefined loops like this one are very tightly coupled to the structure or values of the data that they are working on: the number of items in a list, the number of rows in a matrix or, as in this example, the number of positive integers less than or equal to a particular value.

In such cases it is very often possible to express the entire algorithm in a very straightforward way, without any explicit loops. Usually the result is a much shorter program that is much easier to read and which runs considerably faster than the solution using explicit loops.

For example, in the example above it is possible to replace the loop by a vector of possible divisors produced by the *Index generator*. The algorithm is unchanged, but the program is shorter:

In [151]:
]dinput
DivisorsNoLoops ← {
    (0=(⍳⍵)|⍵)/⍳⍵
}

In [152]:
DivisorsNoLoops 3219

1 3 29 37 87 111 1073 3219

and about 100 times faster:

In [153]:
]runtime -c 'Divisors 3219' 'DivisorsNoLoops 3219'

Divisors 3219 → 3.1E¯3 | 0% ⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕ 
 DivisorsNoLoops 3219 → 3.8E¯5 | -99%

The `]runtime -c` is a user command that can be used to compare two or more pieces of code. It uses the first expression as the baseline and then compares the run time of the subsequent pieces of code against the first one. Here, the `-99%` on the second line means `DivisorsNoLoops` is 100 times faster, which can also be understood by comparing the run times (in seconds) of each expression, the `1.1E¯3` and `1.1E¯5` above (the exact numbers might fluctuate, depending on factors like how the CPU is feeling, the machine the code gets ran on, etc.)

Of course, sometimes the processing that is to take place inside the loop is so complex that it is infeasible to rewrite the program so that it doesn't use an explicit loop. For example, the existence of a dependency such that the calculations taking place in the $n^\text{th}$ iteration are dependent on the results produced in the $(n-1)^\text{th}$ iteration will generally make it harder to write a program without an explicit loop.

### Conditional Loops

In the previous section we used the term "Predefined loops" because the number of iterations was controlled by an expression executed before the loop starts. It is also possible to program loops which are repeated until a given condition is satisfied.

Two methods are available:

 - using `:Repeat... :Until`;
 - using `:While... :EndWhile`.
 
The two methods are similar, but there are some important differences:

 - `:Repeat`:
   - when the loop is initialised, the condition is **not** yet satisfied (generally);
   - the program loops *until* this condition **becomes** satisfied;
   - the "Loop or Stop" test is placed at the **bottom** of the loop;
   - the instructions in the loop are executed at least once.
 
 - `:While`:
   - when the loop is initialised, the condition **is** (generally) satisfied;
   - the program loops as long as it **remains** satisfied;
   - the "Loop or Stop" test is placed at the **beginning** of the loop;
   - the instructions in the loop may not be executed at all.

#### Bottom-Controlled Loop (`:Repeat`/`:Until`)

The control variables involved in the test are often initialised before the loop begins, but they can be created during the execution of the loop because the test is placed at the bottom.

Then the block of statements delimited by `:Repeat... :Until` is executed repeatedly up to the point where the condition specified after `:Until` becomes satisfied.

This condition may involve one or more variables. It is obvious that the statements contained in the loop must modify some of those control variables, or import them from an external source, so that the condition is satisfied after a limited number of iterations. This is the programmer's responsibility.

Here is an example usage of `:Repeat... :Until` to calculate after how many years an investment yields over 500 (of some currency) in interest at a specific rate:

In [154]:
∇ years ← rate ComputeGrowthTime amount
    years ← 0
    :Repeat
        amount ← amount+amount×rate  ⍝ accumulate interest
        years ← years+1
    :Until 500<amount×rate
∇

In [155]:
0.02 ComputeGrowthTime 10000

47

In [156]:
0.02 ComputeGrowthTime 20000

12

This means that if you had 10.000 invested, yielding 2% interest that would then add to your initial investment, at the 47th year the interest alone would be greater than 500. If you start your investment with 20.000 instead, it will only take 12 years.

The test is made on the bottom line of the loop, immediately after `:Until`, so the loop is executed at least once.

In our example, this means that if you start with a large amount of money and/or the rate is very generous, the reported answer is correct: you would get interest greater than 500 right on the first year

In [157]:
0.5 ComputeGrowthTime 1000000

1

The "Loop or Stop" control is made at the bottom of each loop, but it is also possible to add one or more intermediate conditions which cause an exit from the loop using a `:Leave` clause or a *branch arrow* (this will be explained in [the subsection "Modern and Traditional Controls Cooperate"](#Modern-and-Traditional-Controls-Cooperate)).

<!--begin special case style=remark-->
***Special case***:

 > It is possible to replace `:Until` with `:EndRepeat`. However, because there is no longer a loop control expression, the program would loop endlessly. For this reason it is necessary to employ intermediate tests to exit the loop when using this technique.
<!--end-->

#### Top-Controlled Loop (`:While`/`:EndWhile`)

Because the test is now placed at the top of the loop, control variables involved in the test must be initialised before the loop begins.

Then the block of statements limited by `:While`/`:EndWhile` will be executed repeatedly as long as the condition specified after `:While` remains satisfied.

This condition may involve one or more variables. It is obvious that the statements contained in the loop must modify some of those control variables so that the condition is satisfied after a limited number of iterations. This is the programmer's responsibility.

As an example, let us implement a function that takes a positive integer and computes its ["Collatz path"](https://en.wikipedia.org/wiki/Collatz_conjecture). The Collatz path of a positive integer is built by starting at the number and then:

 - if the number $n$ you are at is odd, go to $3n + 1$;
 - if the number $n$ you are at is even, go to $\frac{n}{2}$.
 
The [Collatz conjecture](https://en.wikipedia.org/wiki/Collatz_conjecture) states that any positive integer eventually goes to 1. This tradfn can help us visualise this:

In [158]:
∇ path ← CollatzPath n
    path ← 1⍴n
    :While path[≢path]≠1
        n ← path[≢path]
        n ← ((n÷2) (1+3×n))[1+2|n]
        path ← path,n
    :EndWhile
∇

In [159]:
CollatzPath 1

1

In [160]:
CollatzPath 2

2 1

In [161]:
CollatzPath 11

11 34 17 52 26 13 40 20 10 5 16 8 4 2 1

The test is made in the top line of the loop, immediately after `:While`, so it is possible that the block of statements inside the loop will never be executed. In our case, when `CollatzPath` is called with 1 as its argument.

The "Loop or Stop" control is made at each beginning of a new iteration but it is also possible to add a second control at the bottom of the loop, replacing `:EndWhile` with a `:Until` clause, just like we did for the `:Repeat` loop.

### Exception Control

#### Skip to the Next Iteration (`:Continue`)

In any kind of loop (`:For`-`:EndFor`/`:Repeat`-`:Until`/`:While`-`:EndWhile`) this clause indicates that the program must abandon the current iteration and skip to the next one.

 - In a `:For`-loop, this means that the next value(s) of the control variable(s) are set, and the execution continues from the line immediately below the `:For`-statement.
 
 - In a `:Repeat`-loop, this means that the execution continues from the line immediately below the `:Repeat`-statement.
 
 - In a `:While`-loop, this means that the execution continues from the line containing the `:While`-statement.
 
For the `:For` loop, the execution only proceeds inside the loop if there are any more values for the control variable(s) to take. Similarly, we only execute more iterations of the `:Repeat` and `:While` loops if their conditions still allow.

Consider this tradfn:

In [162]:
∇ r ← ContinueInsideWhile r
    :While r>0
        ⎕← 'start of the loop'
        ⎕← r
        r ← r-1
        :Continue
        ⎕← 'bottom of the loop'
        r ← ¯3
    :EndWhile
∇

In [163]:
ContinueInsideWhile 2

start of the loop
2
start of the loop
1
0

Because of the `:Continue` clause, we never hit the second print and the assignment that would set `r` to -3. When `r` is decreased to 0 and we hit the `:Continue` clause, the control expression in front of the `:While` is evaluated and because `0>0` evaluates to 0, the whole `:While` loop is finally finished.

Usually, `:Continue` statements are nested within other control structures so that we only skip a part of the loop if certain conditions are met.

#### Leave the Loop (`:Leave`)

This clause causes the program to stop the current iteration and to skip any future iterations, aborting the loop immediately, and continuing execution from the line immediately below the bottom end of the loop.

`:Leave` works with any kind of loop.

#### Jump to Another Statement (`:GoTo`)

This clause is used to explicitly jump from the current statement to another one, with the following syntax: `:GoTo destination`.

In most cases, *Destination* is the ***Label*** of another statement in the same program.

A *Label* is a word placed at the beginning of a statement followed by a colon. It is used as a reference to the statement. It can be followed by an APL expression, but for readability it is recommended that you put a label on a line of its own. For example, in

```APL
Next:
val ← goal-val÷2
```

`Next` is a *Label*. `Next` is considered by the interpreter to be a variable whose value is the number of the line on which it is placed. It is used as a destination point both by the traditional *Branch arrow* and by the `:GoTo` clause, like this: `:GoTo Next` is equivalent to `→Next` (cf. [a later section](#Traditional-Flow-Control)). Be careful not to include the colon after the branch name when referencing to it after a `:GoTo` or after a `→`.

##### Jump Destinations

The following conventions apply to the *Destination* of a jump:

| `destination` | Behaviour of executing `:GoTo destination` or `→destination` |
| :-: | :- |
| Valid label | Skip to the statement referenced by that label. |
| `0` | Quit the current function and return to the calling environment. |
| `⍬` | Do not jump anywhere but continue on to the next statement. |

#### Quit This Function (`:Return`)

This clause causes the tradfn to terminate immediately and has exactly the same effect as `→0` or `:GoTo 0`. Control returns to the calling environment.

### Endless Loops

Whatever your skills you may inadvertently create a function which runs endlessly. Usually this is due to an inappropriate loop definition.

However, sometimes APL may appear to be unnecessarily executing the same set of statements again and again in an endless loop, when in fact it just has to process a very large amount of data or perform some heavy calculations.

Fortunately you can interrupt the execution of a function using two kinds of interrupts: weak and strong. Let us see what this means.

#### A Slow Function

Let us consider the function below:

In [164]:
∇ Endless; i
    i ← 0
    :Repeat
        ⎕DL 3
        ⊢i ← i+1
    :Until i=20
∇

This function is not really endless, but the line `⎕DL 3` makes it DeLay all execution for (approximately) 3 seconds, so the whole loop should take about a minute to finish execution. (You will learn more about `⎕DL` and related functions later down the road.)

The line `⊢i ← i+1` has the final `⊢` so the iteration number is printed as the loop progresses. However, if you are running the Jupyter Notebook version of this chapter, the numbers only show up after the iterations are all complete. This is due to a current shortcoming of the Dyalog APL kernel.

**<center>Don't run this function yet!</center>**

#### Weak and Strong Interrupts

If you issue a ***Weak interrupt***, the computer will complete the execution of the statement that it is currently processing. Then it will halt the function before executing the next statement. We recommend using a weak interrupt whenever possible because it allows the user to restart the function at the point it was interrupted (cf. [the chapter with the first aid kit](./First-Aid-Kit)).

If you issue a ***Strong interrupt***, the computer will complete the execution of the APL primitive that it is currently processing. Then it will interrupt the function before executing the next primitive.

For example, in the `Endless` function above, it could calculate `i+1` and stop before executing the assignment `i ← i+1`. Of course, if the user restarts the statement, it will be executed again in its entirety (it is impossible to resume execution in the middle of a statement).

Note that it is impossible to interrupt the execution of a primitive like `i+1` itself, and sometimes the execution of some primitives may take a long time.

#### How Can You Generate an Interrupt

If you are using the Windows IDE, by going to "Action" &#8680; "Interrupt" you can issue a *Weak interrupt*. Similarly, RIDE's "Action" &#8680; "Weak Interrupt" will do the trick.

In order to issue a strong interrupt, you either

 - go to "Action" &#8680; "Strong Interrupt" if you are using RIDE;
 - if you are using the Windows IDE, go to the system tray, look for the Dyalog APL icon, click it and then select "Strong Interrupt" (cf. <!--figure-->the image below<!--Win_Strong_Interrupt-->).
 
![Issuing a strong interrupt in the Windows IDE](res/Win_Strong_Interrupt.png)

The method explained above for the strong interrupts on the Windows IDE also works for weak interrupts. **Be patient!** If Dyalog APL is doing some heavy calculations for you, there may be a few seconds delay from when you click the APL icon until the menu shown above appears.

After you issue an interrupt (weak or strong) there may be a few more seconds delay before the interrupt actually occurs.

Now, let's test it.

#### First a Weak Interrupt

Start running the function and, after a couple of iterations, issue a weak interrupt as you learned in the previous section.

Sadly, the Jupyter Notebook interface has limited capability for dealing with interrupts, so for the remainder of this explanation you will be presented with some figures to illustrate the process.

If we define the tradfn on the Windows IDE, run it and then hit "Action" &#8680; "Interrupt", what happens is illustrated in <!--figure-->the next figure<!--Win_Endless_Weak_Interrupt-->.

![A weak interrupt issued in the Windows IDE](res/Win_Endless_Weak_Interrupt.png)

Notice the `Endless[4]` message is below the few iteration numbers that were printed, and the stack trace window (that opened at the bottom) has highlighted that same line: line 4. This means that the function has been interrupted just before executing line number 4, and you can be sure that line 3 has finished.

To back out from the interrupted state of execution, press the <kbd>Esc</kbd> key as many times as needed, or execute the command `)Reset`, which will be explained shortly.

#### And Now a Strong Interrupt

Run the function again, and after some time issue a strong interrupt.

The result should be slightly different, as you can see in <!--figure-->the next figure<!--Win_Endless_Strong_Interrupt-->:

![A strong interrupt issued in the Windows IDE](res/Win_Endless_Strong_Interrupt.png)

This time, the message `Endless[3] ⎕DL 3` tells you the line it was executing when it got interrupted, and the caret sign tells you which operation was being carried over. In this case, we did not let the 3 second delay finish.

### Traditional Flow Control

In early versions of APL, a unique symbol - the ***Branch arrow*** (`→`, typed with <kbd>APL</kbd>+<kbd>]</kbd>) - provided the only means to override the order in which statements were executed. Today you should only use this mechanism when maintaining code which is already written in this style.

The *branch arrow* works in exactly the same way as the `:GoTo` clause. `→destination` and `:GoTo destination` are strictly equivalent.

`destination` should **always** be a ***Label***. Remember: a *Label* is a word placed at the beginning of a statement and **followed** by a ***colon***. Specifying a statement number (i.e. `→47`) would become invalid as soon as you add or remove lines before that line number.

#### Equivalence With Modern Flow Control

Using small tricks makes it possible to use `→` to emulate the modern keywords that control the flow of a program. Figuring out how to emulate all the keywords can be an interesting *educational exercise* but those tricks should be avoided in modern code.

As an example, we will see how we can emulate an `:If... :EndIf` clause. For that, you need to remember that jumping to an empty `destination` does nothing (cf. [above](#Jump-Destinations)), so that statements are executed sequentially.

Consider the following program:

In [165]:
∇ money ← price IsItExpensive money
    →(~price≤money)⍴expensive  ⍝[1]
    money ← money-price        ⍝[2]
    expensive:                 ⍝[3]
∇

In [166]:
13 IsItExpensive 20

7

In [167]:
13 IsItExpensive 10

10

The program above uses `(~price≤money)⍴expensive` to emulate an `:If` clause and the `expensive:` line acts as the `:EndIf` clause. If you have enough `money` to buy something that costs `price`, it will return how much money you have left after making the purchase. If you don't have enough money, your money is returned unchanged.

Let us walk through the execution of the two example calls above to see exactly how this works:

 - `13 IsItExpensive 20`:
   - `expensive` is the label that is defined on line 3, so its value is 3 as if it were a regular variable;
   - `price≤money` is `1` because `13≤20` and hence `~price≤money` is `0`;
   - `(~price≤money)⍴expensive` is thus equivalent to `0⍴3` which evaluates to `⍬`;
   - `→⍬` does nothing (as explained in [the appropriate section above](#Jump-Destinations)) and the program proceeds to lines 2 and 3.
   
 - `13 IsItExpensive 10`:
   - `expensive` is the label that is defined on line 3, so its value is 3 as if it were a regular variable;
   - `price≤money` is `0` because `13≤10` is not satisfied, hence `~price≤money` is `1`;
   - `(~price≤money)⍴expensive` is thus equivalent to `1⍴3` which evaluates to `3`;
   - `→3` makes the program jump to line 3, skipping line 2.
   
This kind of conditional jump can be summarised like `→(~condition)⍴ destination`.

Using control structures, we would have written

In [168]:
∇ money ← price SaneIsItExpensive money
    :If price≤money
        ⎕← 'I have enough money'
        money ← money-price
    :EndIf
∇

In [169]:
13 SaneIsItExpensive 20

I have enough money
7

In [170]:
13 SaneIsItExpensive 10

10

or even better, without control structures at all we could write

In [171]:
]dinput
IsItExpensiveDfn ← {
    ⍵-⍺×⍺≤⍵
}

In [172]:
13 IsItExpensiveDfn 20

7

In [173]:
13 IsItExpensiveDfn 10

10

This example above shows that programming without explicit control flow is something that is quite feasible and, often times, very elegant.

In your road to mastering Dyalog APL you will want to train your brain to write programs more like the `IsItExpensiveDfn` above and less like the `SaneIsItExpensive`, and definitely not like the labelled `IsItExpensive`!

#### Multiple Destinations

One can also use `→` with multiple labels on the right, in which case program execution jumps to the *first* label in the vector of labels. Just consider the following dummy program, where we use explicit line numbers so we can print them as the tradfn executes:

In [174]:
∇ MultipleDestinations
    →4 3 2 7
    2
    3
    4
    5
    6
    7
    8
∇

In [175]:
MultipleDestinations

4
5
6
7
8

Try changing the numbers in front of the `→` above and see how the numbers printed change.

### Modern and Traditional Controls Cooperate

It is sometimes convenient to mix modern and traditional flow control in order to simplify the code.

Consider a loop in which we must terminate the execution of the function if the condition `x<3` becomes satisfied. With modern control structures, the function would probably include a

```APL
...
:If x<3
    :Return
:EndIf
...
```

which takes up 3 lines. This might be a bit cumbersome. It is possible to use fewer statements using an explicit branch:

```APL
...
:GoTo (x<3)/0
...
```

or equivalently

```APL
...
→(x<3)/0
...
```

since the branch arrow and the `:GoTo` keyword are equivalent.

Something very similar could be written if we need to leave a loop when a given condition becomes satisfied. For example, instead of

```APL
:Repeat
    ...
    :If cond
        :Leave
    :EndIf
    ...
:EndRepeat
...
```

one could write

```APL
:Repeat
    ...
    →cond/hell
    ...
:EndRepeat
hell:
...
```

<!--begin remark-->
***Remark***:

 > Branching using `→` or `:GoTo` is not recommended as a general tool for programming flow control. It is briefly explained here mainly in order to help you understand existing code and to show that the technique may be useful and feasible in special situations. You should, in general, either use control structures or avoid them altogether with alternative formulations of your algorithm(s).
<!--end-->

<!--begin warning-->
***Warning***:

 > Colons are placed **before** a keyword (e.g. `:For`) but **after** a label (e.g. `hell:` above).
<!--end-->

## Input, Output and Format

Up to now, our functions processed values passed as arguments, and returned results which could be used in an expression, assigned to a variable or displayed. But a function can also get data from other sources and/or it can produce results which are not APL variables, but, for example, printed material or data files. In this section you will learn some useful techniques to write such functions.

This section will also help you use data that you may have already have stored in Excel worksheets or in text files on disk.

### Some Input and Output Methods

Here are some of the most typical methods used by a function to get input data or output results.

| Some Input/Output methods | Input | Output | cf. |
| :- | :-: | :-: | :-: |
| Display a result returned at function completion |  | X | |
| Display intermediate values during execution | | X | [see here](#Displaying-Intermediate-Results) |
| Use or modify a global variable | X | X | [see here](#Using-Global-Variables) |
| Exchange data with a spreadsheet (e.g. Microsoft Excel) | X | X | [see here](#Exchanging-Data-With-an-Excel-Worksheet) |
| Read or write data from/to a file (or a database) | X | X | [see here](#Reading-or-Writing-a-Text-File) |
| Print data on a printer | | X | [see here](#Printing-Results-on-a-Printer) |
| Exchange data with a graphical user interface | X | X | [see here](#Using-a-Graphical-User-Interface) |
| Get data typed by the user on the keyboard | X | | [see here](#Requesting-Values-From-the-Keyboard) |
| Use Internet facilities to get/send data | X | X | [see here](#Fetching-Data-From-the-Web) |

Not all of these possibilities will be explained here; we shall limit our investigations to some simple methods that you may test immediately, using some utility functions we define for you. "I/O" is a common abbreviation for "input and output".

We also provide some text files and spreadsheets that you can use for experimentation:

 - Excel workbooks `xldemo.xls` and `worldsales.xls`;
 - text files `mlk.txt` and `report.txt`.
 
We recommend that you place those files in a reference directory where they will be preserved, and copy them to a test directory where you will be able to make some experiments and modify them at will.

We also suggest that you create a global variable in your WS with the path to that test directory, to avoid repeating it in all the tests you will do.

For example, we will set

In [176]:
filepath ← 'C:/Users/rodri/Documents/Dyalog/MDAPL/res/testfiles/'

We recommend that you set this global variable with the *absolute path* to the folder as it is easier to get an absolute path to work.

In the following sections we will refer to this variable `filepath`.

### Format

In the preceding pages we found that we can display, on the screen, any kind of results: numbers, text or a mixture of numbers and characters in nested arrays. Now we shall try to output data to external media, like graphic interfaces, disk files or printers. Most of those media only accept text. For example, it is impossible to send numbers to a printer: we must first convert them to printable characters.

The APL language includes two such conversion tools: a function named ***Format***, represented by the symbol `⍕` and typed with <kbd>APL</kbd>+<kbd>'</kbd>, and a *System function* named `⎕FMT`.

These facilities will be studied in detail in [a subsequent chapter](./Execute-and-Format-Control) but we will introduce here the basic use of *Format*.

#### Monadic Format

Monadic *Format* converts any array (numbers, characters and nested arrays) into its character representation. The result is exactly the same as what you see when you display the array in the APL session, because in fact, APL internally uses monadic *Format* to display arrays.

 - Character values are not converted: they remain unchanged.
 - Numeric and nested values are transformed into vectors or matrices of characters.
 
Here are some examples:

In [177]:
⍴⍕'album'

5

In [178]:
⊢chemistry ← 3 5⍴'H2SO4CaCO3Fe2O3'

H2SO4
CaCO3
Fe2O3

In [179]:
⍴chemistry

3 5

In [180]:
⍴⍕chemistry

3 5

`'album'` and `chemistry` are left unchanged because they are character arrays.

However, the 3-item numeric vector

In [181]:
⍴52 69 76

3

becomes an 8-item character vector once converted:

In [182]:
⍴⍕52 69 76

8

In [a previous chapter](./Data-and-Variables.ipynb) we used a 2 by 3 nested matrix named `nesMat`:

In [183]:
⊢nesMat ← 2 3⍴'Dyalog' 44 'Hello' 27 (2 2 ⍴ 8 6 2 4) (2 3⍴1 2 0 0 0 5)

┌──────┬───┬─────┐
│Dyalog│44 │Hello│
├──────┼───┼─────┤
│27 │8 6│1 2 0│
│ │2 4│0 0 5│
└──────┴───┴─────┘

Notice this matrix has 2 rows and 3 columns:

In [184]:
⍴nesMat

2 3

Once converted into text, it will become 20 characters wide and it will span 3 rows, because the second row of `nesMat` contained two small matrices:

In [185]:
⍴u ← ⍕nesMat

3 20

In [186]:
u

Dyalog 44 Hello 
 27 8 6 1 2 0 
 2 4 0 0 5

#### Dyadic Format

Dyadic *Format* applies **only** to numeric values. It converts them into text in a format controlled by the left argument, which is made up of two numbers:

 - the first number indicates the number of characters (the with of the output) to be used to represent each numeric value;
 - the second number indicates how many decimal digits will be displayed.
 
Let us make some experiments with the following matrix:

In [187]:
⎕RL ← 73
⊢nm ← (?3 4⍴200000)÷100

21.21 1534.88 375.46 704.5 
1125.14 1963.52 464.45 1438.25
 796.53 1569 157.14 886.59

For example, if we want each number to be right aligned in a field with 8 characters of width and with 2 decimal digits, we'd use

In [188]:
8 2⍕nm

21.21 1534.88 375.46 704.50
 1125.14 1963.52 464.45 1438.25
 796.53 1569.00 157.14 886.59

The result has of course 3 rows and 32 columns (4 times 8 characters):

In [189]:
⍴8 2⍕nm

3 32

Instead, we may prefer to have each number in a cell 6 characters wide, with no decimal digits:

In [190]:
6 0⍕nm

21 1535 375 705
 1125 1964 464 1438
 797 1569 157 887

In [191]:
⍴6 0⍕nm

3 24

<!--begin remark-->
***Remark***:

 > Values are not truncated, but **rounded**.
<!--end-->

As an example, consider the second value of the first row:

In [192]:
nm[1;2]

1534.88

In [193]:
5 0⍕nm[1;2]

1535

<!--begin remark-->
***Remark***:

 > Any attempt to apply dyadic `⍕` to characters will cause a `DOMAIN ERROR`.
<!--end-->

*Format* will be studied in detail in [this chapter](./Execute-and-Format-Control.ipynb).

### Displaying Intermediate Results

During normal execution, most applications do not use the session window (the development environment); all input/output is typically done with more user-friendly interfaces. However, during the development of an application, it may be useful for experimental or debugging purposes to have a function display intermediate results.

This can be accomplished in 2 different ways:

 - inside a tradfn, if the result of an expression is not assigned to a variable it gets displayed;
   - in particular, if you add `⊢` to the beginning of any line, that line will display its value, even if the line is an assignment (e.g. the line `⊢v ← 2×3` will assign 6 to `v` and display it);
   
 - any intermediate result can be displayed by assigning it to the ***Quad*** symbol (`⎕`, typed with <kbd>APL</kbd>+<kbd>l</kbd>), and this assignment can be placed anywhere in the middle of a statement:

In [194]:
v ← (⎕←chemistry[1;],chemistry[3;]),⎕←,chemistry[⎕←⍸chemistry='2']

┌───┬───┐
│1 2│3 3│
└───┴───┘
22
H2SO4Fe2O3

Because we used `⎕←` three times, three values got displayed one after the other, on three successive lines of the screen, in the order that they were evaluated:

 1. &nbsp;&nbsp; First `⍸chemistry='2'`,
 2. &nbsp;&nbsp; then `,chemistry[⍸chemistry='2']`
 3. &nbsp;&nbsp; and then `chemistry[1;],chemistry[3;]`.

We **recommend** that you use this last method, which is the most explicit and which can be easily detected by any text search utility function.

### Using Global Variables

A function can use (as input) or modify (as output) the contents of one or more variables which are global to it. The variables may be completely global, or they may be local to a calling function (so-called *semi-global* variables).

For example, as we suggested earlier, a global variable may contain a path used by dozens of functions in a workspace:

In [195]:
filepath

C:/Users/rodri/Documents/Dyalog/MDAPL/res/testfiles/

It will be possible for any function to use this path to prefix some file names, for example as

In [196]:
⊢file1 ← filepath,'sales.xls'

C:/Users/rodri/Documents/Dyalog/MDAPL/res/testfiles/sales.xls

In [197]:
⊢file2 ← filepath,'customers.txt'

C:/Users/rodri/Documents/Dyalog/MDAPL/res/testfiles/customers.txt

Storing common parameters, like a folder path, in global variables can often be very convenient. For example, it makes it very easy to have the system use another set of files without changing any functions. This technique can for example be used to switch between running on test data and on production data.

Similarly, a function can output values into global variables, which may be used by many other functions sharing the same workspace.

You must be very cautious when using this technique:

 - maintenance of functions using global variables is complex because it is difficult to keep track of the different statements which use or update those variables;


 - if an error occurs, and if several functions can modify these global variables, it may be very difficult to determine which of them had last modified a variable involved in the error;


 - if function execution is interrupted and restarted, global values set before the interruption may conflict with new ones calculated in a different context. For example, if the function increments a global variable in line `[1]` but crashes in line `[2]`, and you restart the function, you will have the global variable incremented twice instead of once, as you had expected. Such errors are *very* nasty, as they can lead to other errors or breakdowns much later in the execution. It can be close to impossible to find the causes of such errors in a systematic way.

Such a technique should be restricted to a limited number of variables, clearly identified and documented by any convenient method: a common name prefix, an automated system of references, etc.

Sometimes when starting an application it is necessary to read a lot of settings from a file and make the settings available to all the programs that constitute the application. In such a situation it would make sense to write a program to read the settings from the file and store them in global variables in the workspace.

<!--begin advice style=hint-->
***Advice***:

 > Whenever possible, favour explicit exchange of values through arguments and results.
<!--end-->

#### Global Constants

Because of the reasons explained above programmers are discouraged from writing new values to global variables, but from the text above it is also clear that global variables can be very useful.

In particular, it is often very useful to use global variables to store important pieces of information that are *constant* throughout the whole execution of your application. This way, these important pieces of information can be accessed from within your functions with ease. It is a common convention (and a convention that is not specific to APL) to name these global constants in all upper case letters and to **not** update its value after the variable is initialised.

As an example, the folder in which our files reside can be declared as a global constant:

In [198]:
FILEPATH ← 'C:/Users/rodri/Documents/Dyalog/MDAPL/res/testfiles/'

Notice that from the language point of view, `FILEPATH` is just like any other variable and it *can* be modified:

In [199]:
FILEPATH ← 2

So it is indeed up to the programmer to respect the convention or not. We will respect the convention and hence we'll redefine `FILEPATH` to its original value:

In [200]:
FILEPATH ← 'C:/Users/rodri/Documents/Dyalog/MDAPL/res/testfiles/'

### Exchanging Data With an Excel Worksheet

Many applications are based on an intelligent partnership between Excel and APL:

 - Some users enter data in a set of worksheets with a predefined structure.
 - APL reads the sheets and processes the data with much greater power, precision and flexibility than Excel is capable of.
 - Finally, APL outputs results into one or more worksheets, in which the user can modify the presentation, define additional simple calculations and produce simple graphs.

To help you explore those techniques, we provide two functions that will help you exchange data with an Excel worksheet:

 - `XLGet` imports data from a worksheet into your WS;
 - `XLPut` exports data to a worksheet.

These functions have been developed for learning purposes and you can use them for light applications.

The "loaddata.dws" workspace, distributed as part of the Dyalog APL system, contains a set of professional grade functions to read from and write to text files, Excel workbooks, SQL databases and XML files.

"loaddata.dws" contains the functions `LoadXL` and `SaveXL`, which are similar to `XLGet` and `XLPut`, but with a slightly different syntax.

#### Importing Data

`XLGet` will be used with the following syntax: `XLGet fileID sheet range`, where

 - `fileID` is the full path and name of the workbook, e.g. `'c:/mydir/subdirectory/mydata.xls'`;

 - `sheet` is the name of the worksheet inside the workbook, for example `'Sales'`. If left empty, the function will read the active worksheet;

 - `range` is the part you want to gett, specified using Excel notation, for example `'B4:H8'`. If empty, the function will read all the contents of the selected sheet.

We will now define `XLGet`. Don't worry if you do not understand everything that is going on, you are not expected to. It is common for programmers to be exposed to other people's programs, and looking at them for the first time can be daunting and confusing at first. Usually, this "fear" and confusion degrades gracefully as you become more acquainted with the code you are looking at, until you eventually understand the code well enough for whatever purposes you need it.

It is better to start going through experiences like this sooner than later, so here is the `XLGet` function:

∇ r ← XLGet d; shAct; shNames; shIndex; xl; range; sheet; book; msg                                    
  ⍝ D     : 'Workbook' 'Sheet' 'Range'                                                          
  ⍝ Range : Should be specified with traditional Excel conventions (for example 'B4:G7')        
  ⍝       | If empty, read all                                                                  
  ⍝ Sheet : Character string (for example '2010')                                               
  ⍝       | If empty, read the active worksheet                                                 
                                                                                               
  (book sheet range) ← 3↑d,'' ''                                                             
  :Trap 0                                                                                      
      xl ← ⎕NEW'OleClient'(⊂'ClassName' 'Excel.Application')     ⍝ Start Excel                   
      xl.Application.Visible ← 1                                 ⍝ Make it visible for demos     
  ⍝------------------------------------------------------------⍝ Open an existing workbook.    
      xl.Workbooks.Open(⊂book)                               ⍝ Be careful: enclose FileId    
  ⍝------------------------------------------------------------⍝ Activate the appropriate sheet
      :If 0<≢sheet                                           ⍝ "Sheet" has been specified    
          shNames ← xl.ActiveWorkbook.(⌷Sheets).Name             ⍝ Names of all the sheets       
          shIndex ← shNames⍳⊂sheet                             ⍝ Search the correct one        
          xl.ActiveWorkbook.Sheets.Item[⊂shIndex].Activate     ⍝ Activate it                   
      :EndIf                                                                                   
      shAct ← xl.ActiveWorkbook.ActiveSheet                      ⍝ Select the active sheet       
  ⍝------------------------------------------------------------⍝ Read the contents             
      :If 0=≢range
          r ← shAct.UsedRange.Value2                             ⍝ Read the whole sheet          
      :Else                                                                                    
          r ← shAct.Range[⊂range].Value2                       ⍝ Read just a portion           
      :End                                                                                     
      ((,r∊⎕NULL)/,r) ← ' '                                      ⍝ Whiten empty cells            
  ⍝------------------------------------------------------------⍝ Get out of that               
      xl.ActiveWorkbook.Close 0                                ⍝ Close the book                
      xl.Quit                                                  ⍝ Close XL                      
  :Else                                                                                        
      r ← ''                                                                                     
      ⍝'Msg'⎕WC'MsgBox' 'Unable to read this Worksheet:'book('Event' 61 1)                    
      ⍝⎕DQ'Msg'
      ⎕← 'Unable to read this worksheet: ', book
  :EndTrap                                                                                     
∇

⊢val1 ← XLGet (FILEPATH,'WorldSales.xls') 'Madrid' ''

In this example we get all the contents of the `'Madrid'` sheet, because no `range` was specified. Instead, we can get just the table header and the first three rows of the active sheet, by specifying the `range` and by omitting the sheet:

⊢var2 ← XLGet (FILEPATH,'WorldSales.xls') '' 'A2:D5'

As one cannot predict which was the active worksheet when the workbook was last saved, it is best to only provide an empty `sheet` argument for single-sheet workbooks.

#### Exporting Data

`XLPut` has the following syntax: `values XLPut fileID sheet startCell`, where

 - `values` are the values to write to the target worksheet;
 - `fileID` has the same meaning as in `XLGet`;
 - `sheet` is the name of an existing worksheet in that workbook. It will be modified. If empty, a new worksheet will be inserted, with a default name;
 - `startCell` is the top left cell to be written; for example, `'E5'`. The function will automatically deduce the corresponding `range` from the `startCell` and `⍴values`.

Here is the implementation of the function:

### Reading or Writing a Text File

Files can have many different formats. Some contain integer or decimal numbers, some contain APL variables (vectors, arrays, nested values) as we shall see in [a future chapter](./File-Processing.ipynb), but most files contain plain text.

In this chapter, we shall limit our investigations:

 - to text files which can be viewed and modified using a basic text editor like Microsoft Notepad;
 - to rather small files that can be read or written as a whole in a single operation (depending on your workspace size, you can easily read/write thousands of lines of text).

Such a file can be considered as a long vector containing special line separation characters ("*carriage return*" and "*line feed*"). For that reason, when edited with Notepad, they look more or less like APL character matrices.

Except for special purposes, to exchange data between APL and a text file, one needs to:

 - convert the read data from a file as a character vector into a more convenient APL character array (to read), or
 - convert an APL character array into a special character vector with embedded line separators (to write).

Those conventions require some techniques we haven't seen yet. This is the reason why we give you some predefined functions. You will find them in a workspace named `Files.dws` delivered with Dyalog APL. It contains a *Namespace* (you can read about namespaces in [their chapter](./Namespaces.ipynb)) which itself contains the functions. To use them, you must:

 - copy the *Namespace* inside `Files.dws` into your active workspace (be careful with the case):

In [201]:
)copy files Files

C:\Program Files\Dyalog\Dyalog APL-64 18.0 Unicode\ws\files.dws saved Wed Jul 15 20:03:22 2020

 - set an access path to the *Namespace* contents:

In [202]:
⎕Path ← 'Files'

Here are 3 functions you might find useful:

 - `ReadAllText` reads the contents of a text file and returns a character vector. The lines of the text are separated by two special characters: *Carriage Return* and *Line Feed*.
   - Syntax: `result ← ReadAllText path,fileID`

In [203]:
⊢dream ← ReadAllText FILEPATH,'mlk.txt'

So I say to you, my friends, that even though we must face the 
difficulties of today and tomorrow, I still have a dream. It is 
a dream deeply rooted in the American dream that one day this 
nation will rise up and live out the true meaning of its creed: 
we hold these truths to be self-evident, that all men are 
created equal. 
I have a dream that one day, on the red hills of Georgia, sons

 - `ReadAllLines` reads the contents of a text file and returns a nested vector of character vectors, one vector per line of text in the file.
   - Syntax: `result ← ReadAllLines path,fileID`

In [204]:
⊢dreamLines ← ReadAllLines FILEPATH,'mlk.txt'

┌──────────────────────────────────────────────────────────────┬───────────────────────────────────────────────────────────────┬─────────────────────────────────────────────────────────────┬───────────────────────────────────────────────────────────────┬─────────────────────────────────────────────────────────┬──────────────┬──────────────────────────────────────────────────────────────┬──────────────────────────────────────────────────────────────┬──────────────────────────────────────────────────┬─────────────────────────────────────────────────────────────┬────────────────────────────────────────────────────────────────┬────────────────────────────────────────────────────────────────┬────────────┬───────────────────────────────────────────────────────────┬──────────────────────────────────────────────────────────────┬──────────────────────────────────┬─────────────────────┬──────────────────────────────────────────────────────────────┬────────────────────────────────────────────────────────────┬──────────────────────────────────────────────────────────────┬────────────────────────────────────────────────────────────────┬───────────────────────────────────────────────────────────────┬───────────────────────────────────┬────────────────────────────────────────────────────────────────┬──────────────────────────────────────────────────────────────┬─────────────────────────────────────────────────────────────────┬──────────────────────────────────────────────────────────────┬─────────┬───────────────────────────────────────────────────────────────┬─────┬───────────────────────────────────────────────────────────────┬───────────────────────────────────────────────────────────┬──────────────────────────────────────────────────────────────┬────────────────────────┬───────────────────────────────────────────────────────────────┬───────────────────────────────────────────────────────────────┬─────────────────────────────────────────────────────────────┬────────────────────────────────────────────────────────────────┬─────────────────────────────────────────────────────────────────┬─────────────────────────────────────────────────────────────────┬──────────────────────┬──────────────────────────────────────────────────────────────────┬───────────────────────────────────────────────────────┬──────────────────────────────────────────────────────────────────┬──────────────────────────────────────────────────────────┬──────────────────────────────────────────────────────────┬──────────────────┬────────────────────────────────────────────────┬───────────────────────────────────────────────────┬──────────────────────────────────────────────────────────────────┬─────────────────────────────────────┬─────────────────────────────────────────────────────────────────┬──────────────────────────────────────────────────────────────┬───────────────────────────────────────────────────────────────┬─────────────────────────────────────────────────────────────────┬────────────────────────────────────────────────────────────────────┬────────────────────────────────────────────────────────────────────────┬─────────────────────────────┬────────────────────────────┬┐
│So I say to you, my friends, that even though we must face the│difficulties of today and tomorrow, I still have a dream. It is│a dream deeply rooted in the American dream that one day this│nation will rise up and live out the true meaning of its creed:│we hold these truths to be self-evident, that all men are│created equal.│I have a dream that one day, on the red hills of Georgia, sons│of former slaves, and sons of former slave-owners will be able│to sit down together at the table of brotherhood. │I have a dream that one day, even the state of Mississippi, a│state sweltering with the heat of injustice, sweltering with the│heat of oppression, will be transformed into an oasis of freedom│and justice.│I have a dream that my four children will one day live in a│nation where they will not be juged 

 - `PutText` writes a character matrix (or vector) to a file. If a file with the same name already exists, it is replaced by the new file. The function returns the number of characters written to the file.
   - Syntax: `number ← textMatrix PutText path,fileID`

In [205]:
⊢number ← chemistry PutText FILEPATH,'chemistry.txt'

17

The workspace `loaddata.dws`, distributed as part of the Dyalog APL system, contains a set of professional grade functions to read from and write to text files, Excel workbooks, SQL databases and XML files

`loaddata.dws` contains two functions `LoadText` and `SaveText` to complement the set of functions described above. `LoadText` and `SaveText` are designed to work with comma separated files (.CSV files) and fixed field width text files.

We have only used text files. Dyalog APL also includes an advanced file system designed to work very easily and efficiently with APL arrays, as well as a generalised interface to SQL databases. The APL file system is described in [its own chapter](./File-Processing.ipynb). The SQL interface is not described in this tutorial; please refer instead to the specialised brochures.

### Printing Results on a Printer

Missing

### Using a Graphical User Interface

Missing

### Requesting Values From the Keyboard

Here again, if the user of an application is required to enter some input data, this is often done via some user-friendly GUI interface like the previous one or from a web page. However, during the development phase or for a light application it may be simpler to use a very basic question/answer mode.

Two symbols are used to request input from the user:

 - ***Quad*** (`⎕`) is used to enter one or more numbers.
 - ***Quote-Quad*** (`⍞`, typed with <kbd>APL</kbd>+<kbd>Shift</kbd>+<kbd>[</kbd>) is used to enter any string of characters.

#### Quad Evaluated Input

The first method is no longer in common use, but can be useful when prototyping an application which needs to request input. Also, you can easily accidentally type `⎕` in the session and activate Quad evaluated input, so it is worth studying it briefly.

***Quad*** causes a pause in the execution of a function and allows the user to type any kind of expression. That expression is *evaluated* like any APL expression entered in the session, and its *result* is the result of the ***Quad*** function.

 - When a function executes a *Quad*, it displays `⎕:` at the left margin of the screen to inform the user that he or she is expected to enter one or more values.

 - If any error occurs during evaluation, the input *Quad* is displayed over and over again, until the user succeeds in entering a valid expression.
 
In the following example, a function is supposed to count how many items of its right argument there are between two limits. These limits could be passed via a left argument, but in this case we choose to ask the user to enter them during execution of the function:

In [206]:
∇ Prompt1 vector; lim1; lim2; nb
    'What are the limits?'
    (lim1 lim2) ← ⎕
    nb ← +/(vector>lim1)∧(vector<lim2)
    (⍕nb),' values are comprised between ',(⍕lim1),' and ',⍕lim2
∇

As of the time of writing, Jupyter notebooks do not support input through `⎕` (nor through `⍞`). We do encourage *you* to try these examples out for yourself somewhere other than the Jupyter Notebook interface.

```APL
    Prompt1 salaries
What are the limits?
⎕:
    1000 2000
4 values are comprised between 1000 and 2000
```

You can use this `salaries` variable:

In [207]:
⎕RL ← 73
⊢salaries ← ?20⍴5000

2121 4778 4914 3139 4561 221 1252 1530 4475 332 3507 1829 1710 4964 3849 669 3653 3415 3867 151

Let us make an error while executing the function again:

```APL
      Prompt1 salaries
What are the limits?
⎕:
      mylims
VALUE ERROR: Undefined name: mylims
      mylims
      ∧
⎕:
      ⌊0.5+0.9 1.1×Average salaries    
0 values are comprised between 2472 and 3022
```

Notice that we tried to answer with a variable name but that variable does not exist. An error message is issued, which is not very user-friendly! Then the user is automatically requested again to provide a value.

Then we provide a valid value with an expression that makes use of a variable, a defined function and some primitives. The expression evaluates to `2472 3022` and those are the limits used.

<!--begin hint-->
***Hint***:

 > If you accidentally start evaluated input in the session, type <kbd>Space</kbd> and then hit <kbd>Enter</kbd> to interrupt the input loop.
<!--end-->

#### Quote-Quad Character Input

***Quote-Quad*** allows the user to enter a string of characters. That string is returned as the result of the ***Quote-Quad*** function.

If the user types nothing and just presses the <kbd>Enter</kbd> key, the returned value is an empty text vector.

In [208]:
∇ Prompt2; tex
    'Type any string of characters'
    tex ← ⍞
    '"',tex,'" is a vector of length ',⍕≢tex
∇

```APL
     Prompt2
Type any string of characters
Are you serious?
"Are you serious?" is a vector of length 16
```

You can see that the *Quote-Quad* does not display any prompt sign.

<!--begin advice-->
***Advice***:

 > Avoid using *Quad* evaluated input, as users can inadvertently (or intentionally) interfere with your own function variables.
 >
 > *Quote-Quad* remains useful for quick tests, the development of light interactive functions or scripting applications which work with redirected I/O.
<!--end-->

## Syntax Considerations

### Comments and Statement Separators

#### Comments

It is possible to write comments in a function to make it easier to read, explain how it should be used and help future maintenance or improvements.

Comments being with the symbol ***Lamp*** `⍝` (because it "illuminates" the code). *Lamp* can be typed with <kbd>APL</kbd>+<kbd>,</kbd>.

They can be placed alone on a dedicated line or to the right of any statement, including the function header. All the characters placed to the right of the *Lamp* are ignored by the interpreter.

<!--begin advice-->
***Advice***:

 > Determining when and where to comment your code is subjective but a good piece of advice is to try and **not** use comments to spell out *what* the code is doing; instead, consider using comments to explain *why* the code is doing what it is doing.
<!--end-->

#### Statement Separators

Several statements can be placed on the same line of a function, using the statement separator ***Diamond*** `⋄`, which can be typed with <kbd>APL</kbd>+<kbd>`</kbd>. Statements are executed **starting from the leftmost one**.

Putting several statements on the same line does not save computing time and generally does not improve the readability of a function. It should be reserved for short, straightforward statements.

The diamond separator can also be used in ***Multithreaded programming*** to force the execution of a set of statements without any switch to another thread. But this is outside the scope of this tutorial.

As an example of the usage of comments and statement separators, consider a revision of the `Interlace` function we have seen above:

In [209]:
∇ r ← a Interlace b; size; even  ⍝ This is just a demo
    ⍝ a & b are matrices of the same shape
    size ← 1 2×⍴a     ⍝ the final result has twice as many columns as a
    r ← size⍴0
    even ← 2×⍳(⍴b)[2] ⍝ final indices of the columns of b
    r[;even] ← b ⋄ r[;even-1] ← a ⍝ interlace
∇

We have placed some comments: in the header, on a dedicated line and to the right of some statements. We have also grouped the last two statements on a single line, separated by a *Diamond*.

### Why Should a Function Return a Result?

During the execution of a tradfn any result which is not assigned to a variable name is immediately displayed on the screen; we used that in our functions `Prompt1` and `Prompt2` above.

So, let us compare two very similar tradfns and their usage:

In [210]:
∇ z ← x Plus y
    z←x+y
∇

In [211]:
6 Plus 8

14

In [212]:
∇ x PlusNoRes y
    x+y
∇

In [213]:
6 Plus 8

14

Apparently, both functions work very well:

 - the left one calculates a local variable `z`, which has been declared to be the function result. The function returns the value of `z` as its result, and because the result is not assigned or used, it is displayed on the screen.
 
 - the right one calculates `x+y`. Because this sum is not assigned, it is immediately displayed.
 
 But now, let us try to include these functions in more complex expressions:

In [214]:
10×6 Plus 8

140

In [215]:
10×6 PlusNoRes 8

14
VALUE ERROR: No result was provided when the context expected one
      10×6 PlusNoRes 8
           ∧


Something went wrong!

 - The leftmost function returns a result. The result is available to the calling expression, in this case as the right argument to the multiply function, so that we obtained the answer we wanted.
 
 - But the rightmost function returns no result. The value calculated in the $1^\text{st}$ statement is just displayed (though we did not need it); it is **not** returned as a result. So, the multiply function has an argument on its left (the 10) but nothing on its right, hence the error message.

<!--begin advice-->
***Advice***:

 > Whenever you can, write functions which produce an explicit result. You can always throw the result away if you don't need it.
<!--end-->

This is not even a problem in dfns, because dfns *always* return an explicit result, or they don't return at all. Because dfns are the more modern way of writing user-defined functions and because we encourage you to write dfns whenever possible, you are likely to not have to worry about this.

### Different Types of Functions

#### What Is an Explicit Result?

We have seen that some functions (like `Average` or `Plus`) return a result; we describe the result as being ***Explicit***. It means that once the function has been executed, the result is passed on to the next part of the expression which is being evaluated or, if there is none, it is displayed on the screen.

Some other functions (like `PlusNoRes`) do not return an explicit result. This does not mean that they do nothing; perhaps they read or write data from/to a file or an Excel worksheet, perhaps they print a graph on a printer, or perhaps they build a graphic user interface. All these consequences which arise from the execution of the function can be called ***Implicit*** or hidden results. Anything that happens during the execution of a function and that is not communicated directly in the function's result is generally called a *side effect* of the function (and perhaps its sub-functions).

#### Six Major Types of Functions (Valence)

The number of arguments a function takes is termed its ***Valence***.

You have already met functions with one or two arguments (Monadic or Dyadic), you can also write functions which take no arguments at all: they are called ***Niladic***. Though they do not receive values through arguments, they can process data introduced via the various techniques described in [the previous section on I/O](#Input,-Output-and-Format). Notice that only tradfns can be niladic, as dfns take *at least* one argument.

Depending on whether or not they return an explicit result, tradfns can be classified as follows:

| Valence | With an explicit result | Without an explicit result |
| :- | :- | :- |
| Niladic | `∇ z ←   function` | `∇     function` |
| Monadic | `∇ z ←   function x` | `∇     function y` |
| Dyadic | `∇ z ← x function y` | `∇   x function y` |

Dfns, on the other hand, can only be of two different types: either monadic with an explicit result or dyadic with an explicit result.

Niladic functions which return no result are very similar to scripts written in other programming languages.

Some of the functions which we have already written can be classified as follows:

| Syntax | With result | Without result |
| :- | :-: | :-: |
| Niladic |  | `Endless` |
| Monadic | `Average`, `HarmonicMean` | `Prompt1`, `Prompt2` |
| Dyadic | `Interlace`, `Plus`, `Times` | `PlusNoRes` |

#### Ambivalent Functions

Most APL symbols are used to represent both monadic and dyadic primitive functions. For example, the symbol `⍴` represents both the *Shape* (`⍴y`) function and the *Reshape* (`x⍴y`) function, and `⌈` represents both *Ceiling* (`⌈y`) and *Maximum* (`x⌈y`). These symbols are said to be ***Ambivalent***.

In [this previous section](#Default-Left-Argument) we saw how to write an *ambivalent* dfn. To write an *ambivalent* tradfn, the name of the left argument in the header is specified within braces (to show that is is optional), like this: `∇ result ← {left} Function right`.

We are now faced with a problem: this function must work correctly whether or not a left argument is provided. One of the ways to deal with this in a dfn is by specifying a default left argument. There is another way that works for both dfns and tradfns, and then there is an additional, preferred, way of dealing with this ambiguity in tradfns.

A way to test for the presence of the left argument is through the *System function* `⎕NC`. NC stands for ***Name Classification***, as that is what the `⎕NC` function does: classifies the use that is being given to a given name.

As we shall see in [a future chapter](./System-Interfaces.ipynb),

 - the *Name Classification* of a variable name is 2;
 - the *Name Classification* of a function name is 3;
 - the *Name Classification* of an unused name is 0.

As some examples, recall that `'chemistry'` is the name of a variable, `'Average'` is the name of a function and so far we haven't used the name `'thingamabob'` for anything:

In [216]:
⎕NC 'chemistry'

2

In [217]:
⎕NC 'Average'

3

In [218]:
⎕NC 'thingamabob'

0

So, we can use the expression `0=⎕NC'⍺'` to check if a dfn was called monadically and, in the example above, use `0=⎕NC'left'` to check if the tradfn was called monadically. Conversely, we can use `2=⎕NC'⍺'` to check if a dfn was called dyadically and, in the example above, use `2=⎕NC'left'` to check if the tradfn was called dyadically.

Just to check that it works, let us write a useless ambivalent dfn:

In [219]:
]dinput
Useless ← {
    0=⎕NC'⍺': 'monadic'
    2=⎕NC'⍺': 'dyadic'
    'uh, what..?'
}

In [220]:
23 Useless 78

dyadic

In [221]:
Useless 71

monadic

Can you see why this `⎕NC` method is better suited for dfns than for tradfns?

In a dfn, the left argument is *always* called `'⍺'`, whereas for a tradfn it can change. If you change the name of the left argument in the header of the function and forget to change its name in the `⎕NC` test you can get into trouble. For that matter, it is recommended that you use a different method to check if a tradfn was called monadically:

The ***900 I-Beam***, or `900⌶` (where `⌶` is the ***I-Beam*** character, typed with <kbd>APL</kbd>+<kbd>Shift</kbd>+<kbd>1</kbd>). The *I-Beam* will be covered in [more detail](./Operators.ipynb), here we ask you to close your eyes and just believe `900⌶` does what we tell you it does.

When used inside a tradfn, `900⌶⍬` returns 1 if the tradfn was called monadically and 0 otherwise.

#### Example Ambivalent Function

The following tradfn rounds a numeric value to its $n^\text{th}$ decimal digit:

In [222]:
∇ r ← n Round val; t
    t ← 10*n
    r ← (⌊0.5+val×t)÷t
∇

In [223]:
2 Round 41.31875 82.92413 127.71625

41.32 82.92 127.72

In [224]:
2 0 3 Round 41.31875 82.92413 127.71625

41.32 83 127.716

Now, suppose that we usually want to round values to the second decimal digit: we can decide that if we do not specify a left argument this will be the default behaviour of our function. Here is the updated version of the function:

In [225]:
∇ r ← {n} Round val; t
    :If 900⌶⍬
        n ← 2    ⍝ default value if Round is called monadically
    :EndIf
    t ← 10*n
    r ← (⌊0.5+val×t)÷t
∇

Now the left argument is optional and an `:If... :EndIf` clause detects its presence. If absent `n` is set to 2. The expressions shown below illustrate the two ways in which the function can be used. This function really is ambivalent.

In [226]:
2 0 3 Round 41.31875 82.92413 127.71625

41.32 83 127.716

In [227]:
Round 41.31875 82.92413 127.71625

41.32 82.92 127.72

Alternatively, consider writing a dfn in which providing a default left argument is much cleaner:

In [228]:
]dinput
Round ← {
    ⍺ ← 2
    t ← 10*⍺
    (⌊0.5+⍵×t)÷t
}

#### Shy Result in a Tradfn

Like for dfns (cf. [this](#Shy-Result)), tradfns can return a shy result: in the header of the function we enclose the name of the return variable with `{}`, like so:

In [229]:
∇ {r} ← x ShyFun y
    r ← x×y
∇

In [230]:
17 ShyFun 3

In [231]:
⎕← 17 ShyFun 3

51

In [232]:
10×17 ShyFun 3

510

#### An Argument Used as a Result

It is possible to use the same name for the result as one of the arguments. For example, these are two valid headers: `∇ x ← x Fun y` and `∇ y ← x Fun y`.

This may be useful when a condition causes the function to terminate without any processing. Supposed that you want to repeatedly divide the right argument by 2, until the result becomes odd. A simple loop will do it, but if the argument is already odd, the loop stops immediately and the result is equal to the argument:

In [233]:
∇ y ← DivideIt y
    :While 0=2|y
        y ← y÷2
    :EndWhile
∇

If `y` is even the loop is executed at least once.

If `y` is odd the function exists immediately and returns the unmodified value of `y` as its result.

Of course this is only relevant for tradfns, as dfns do not return the value of a specific variable.

### Nested Argument and Result

#### Nested Right Argument

Both the left and right arguments of a function can be nested vectors, as in `Function 'London' 'UK' 7684700 40`, for example.

A composite argument of this sort is often split (dispatched) into a number of separate local variables using multiple assignment, as illustrated in the following dfn:

In [234]:
]dinput
Dispatch ← {
    (town country population prefix) ← ⍵
    country
}

In [235]:
Dispatch 'London' 'UK' 7684700 40

UK

For tradfns, a more elegant way to achieve the same thing is to specify the composite nature of the argument directly, as follows:

In [236]:
∇ r ← Dispatch (town country population prefix)
    r ← country
∇

In [237]:
Dispatch 'London' 'UK' 7684700 40

UK

Using this syntax, the items of the right argument are automatically allocated into local variables, as you could see above.

<!--begin remark-->
***Remark***:

 > This special syntax applies **only** to the right argument, not to the left one. If a nested vector is passed on the left it can be split (dispatched) by multiple assignment, as suggested earlier.
<!--end-->

#### Choice of Syntax

When a function `Fun` is to receive two values `x` and `y`, you now have the choice between two syntaxes:

 - dyadic `x Fun y`
 - monadic `Fun (x y)`
 
The dyadic way has the advantage that the function can be used with *Reduction*, while the monadic version cannot.

For example, we can use `Plus` in this kind of expression:

In [238]:
Plus/14 10 52 1 12 43

132

#### Nested Result

Similar notation can be applied to the result of the function. Suppose that you want to return a vector of 3 separate (local) variables (named `one`, `two` and `three` in the example) as the result of the function. One way is to declare that the function returns a single named result, and to assign the 3 local variables into the result before the function terminates. Another approach is to simply declare the structure of the result in the header as illustrated below. As these names (`one`, `two` and `three`) appear in the header, they are local.

We exemplify this in the next tradfn:

In [239]:
∇ (one two three) ← left NestedRes right
    one ← left+right
    two ← right[left]
    three ← two×one[1]
∇

In [240]:
3 NestedRes 12 45 78

┌────────┬──┬────┐
│15 48 81│78│1170│
└────────┴──┴────┘

It should be clear from the construct itself, but this special syntax also only applies to tradfns. In dfns, you would have to do something like:

In [241]:
]dinput
NestedRes ← {
    one ← ⍺+⍵
    two ← ⍵[⍺]
    three ← two×one[1]
    one two three
}

In [242]:
3 NestedRes 12 45 78

┌────────┬──┬────┐
│15 48 81│78│1170│
└────────┴──┴────┘

### Choice of Names

Function names (as well as argument and result names, if you are using tradfns) can be any valid APL name, for example `∇ cain ← adam PLUS eve` or `dumb ← man WITHOUT voice`.

However, it is recommended that you use simple names, names that are easy to remember, and names that are consistent from one function to the other. This is especially important if several people have to maintain a common set of functions; any of them should be able to understand immediately which variables represent the arguments and which the result...

This recommendation is really about adopting a strict *naming convention*, and this applies to any serious programming project, whether it uses APL or not.

Here are some simple conventions that you might consider:

 - `∇ z ← x Function y`, often used by English-speaking developers;
 - `∇ r ← g Function d`, often used by French-speaking developers;
 - `∇ r ← a Function b`;
 - etc.

However, we advise you to adopt meaningful words to indicate the nature of the arguments, for example using `numVec` for numeric vectors or `texMat` for character matrices.

We also recommend that you avoid modifying the arguments in the body of the function, otherwise you could make maintenance much more difficult.

## Recursion

A function is ***Recursive*** when it calls itself, generally to calculate the $n^\text{th}$ step of an algorithm from its $(n-1)^\text{th}$ step.

For example, the factorial of $n$ can be defined as $n$ times the factorial of $n-1$. Of course, some condition must specify when the process is supposed to stop. In our example, the factorial of 1 is not calculated, but set to 1. A recursive function could be written as follows:

In [243]:
]dinput
Fact ← {
    ⍵<2: 1
    ⍵×Fact ⍵-1
}

In [244]:
Fact 8

40320

<!--begin warning-->
***Be careful***:

 > *Recursive* solutions are generally very elegant, however:
 >
 > - because a function may call itself a great number of times before it reaches the exit condition, this technique may need a lot of memory if the function works on huge arrays;
 >
 > - when the function calls itself, the variables calculated during one step must not interfere with the calling context. You must carefully localise all your variables if you are using a tradfn -- otherwise this is already done for you. Either way, this is the reason for possible large memory consumption, as the local variables in all recursions may exist at the same time.
<!--end-->

A possible way to overcome some of these limitations is introduced in [the Specialist's Section about Tail Recursion](#Tail-Recursion).

Beware that recursion is never mandatory. A recursive function can always be rewritten using looping or array-oriented techniques instead of recursion.

For example, one could write the dfn that follows:

In [245]:
]dinput
Fact2 ← {
    ×/⍳⍵
}

In [246]:
Fact2 8

40320

Notice that the dfn above is preferred over a tradfn with an explicit loop like a `:For` or a `:While`.

### Recursion in Dfns

Both dfns and tradfns can be recursive by calling themselves explicitly, like you saw above. However, if you are writing a dfn you can also use the special symbol `∇` (the *Del* symbol) to represent this self-call.

Using this symbol in our `Fact` function, we'd write

In [247]:
]dinput
Fact ← {
    ⍵<2: 1
    ⍵×∇⍵-1
}

An implicit self-reference using `∇` needs less interpretative overhead and therefore it may execute more quickly. Moreover, it will continue to work even if the function is renamed.

## Assigning Names to Functions

You can give names to values, through the use of variables like

In [248]:
x ← 2

and you can define functions, like

In [249]:
TimesTwo ← {2×⍵}

but you can also associate a new name with an **existing** primitive, a user-defined function or a derived function. For example,

In [250]:
Rho ← ⍴
2 2 Rho 1 2 3 4

1 2
3 4

In [251]:
Times2 ← TimesTwo
Times2 10

20

In [252]:
Sum ← +/
Sum 1 2 3 4

10

Later on, in [the chapter about Tacit Programming](./Tacit-Programming.ipynb) you will see that you can even assign more primitives to a name, and those will behave in a very specific way.

<!--begin remark-->
***Remark***:

 > When assigning a new name to an existing tradfn, the contents of the original function are not duplicated.
<!--end-->

For example, here is a basic tradfn:

In [253]:
∇ r ← Add3 y
    r ← y+3
∇

And now we assign it a new name:

In [254]:
NewName ← Add3

If we check `NewName`'s code, it will be `Add3`'s source code:

In [255]:
NewName

∇ r←Add3 y
[1] r←y+3 
 ∇

And finally, if we modify `Add3`, `NewName` also gets modified:

In [256]:
∇ r ← Add3 y
    r ← y+30
∇

In [257]:
NewName 10

40

<!--begin tip-->
***Tip***:

 > If you accidentally assign a function to a new name when you wanted to use that name for a value, you can use `)erase` to fix that.
<!--end-->

For example, say you typed this in your interpreter and accidentally pressed <kbd>Enter</kbd>:

In [258]:
Val ← Average

But what you really wanted to type was:

In [259]:
Val ← Average 41 11 19

SYNTAX ERROR: Invalid modified assignment, or an attempt was made to change nameclass on assignment
      Val←Average 41 11 19
      ∧


You can see that gives a `SYNTAX ERROR` because you are trying to change `Val` from a function to a value, and the interpreter doesn't want you to do that.

Instead, first erase `Val` with

In [260]:
)erase Val

and then assign it normally:

In [261]:
Val ← Average 41 11 19

## About the Text Editor

Most of the features of the built-in Dyalog APL editor are very familiar to those of other common editors, but some are very specific and we therefore provide a brief description of the specific editing facilities of Dyalog APL.

### Invoking the Text Editor

Double-clicking a name which represents an existing item invokes the editor and displays its contents, using the colour scheme appropriate for the type of the item (function, character matrix, nested array, etc) defined via "Options" &#8680; "Colors..." if you are using the Windows interpreter or via "Edit" &#8680; "Preferences" &#8680; "Colours" if you are using RIDE.

You can also invoke the editor by pressing <kbd>Shift</kbd>+<kbd>Enter</kbd> when the input cursor is inside or adjacent to the name. This is perhaps the most convenient way as, when working in an APL session, you tend to use the keyboard much more than the mouse.

Let us define a character matrix with the uppercase latin alphabet:

In [262]:
⊢charMat ← 2 13⍴⎕A

ABCDEFGHIJKLM
NOPQRSTUVWXYZ

For some items (e.g. numeric matrices, some nested arrays) the editor is only good for viewing them, while for others such as functions, text vectors and text matrices, the editor can also be used to edit them. In <!--figure-->the example below<!--Win_Editor_Edit_Character_Matrix--> we have invoked the editor, and changed the contents of our `charMat` variable:

![Editing a character matrix with the built-in editor of the Windows interpreter](res/Win_Editor_Edit_Character_Matrix.png)

In <!--figure-->the figure<!--Win_Editor_Edit_Character_Matrix--> the edit window tells us that we modified the character matrix. We must now fix it, like we did with dfns before, for the variable to reflect its new value. RIDE doesn't tell you that the character matrix was modified, but you still need to fix it if you want the new changes to come into effect.

If we now fix the changes to `charMat`, it will become a matrix with 4 rows and 29 columns (the length of its longest row).

If, for some reason, you made a mistake, you can exit the edit window *without* fixing the changes by pressing <kbd>Shift</kbd>+<kbd>Esc</kbd>.

If a name is currently undefined (has no value), double-clicking or pressing <kbd>Shift</kbd>+<kbd>Enter</kbd> on that name invokes the editor on it as if it were a new function. This is one way to create a function.

You can also invoke the editor using the command `)ED` as we did before. By default, it opens a **function** definition, but you can explicitly specify the type of a new object by prefixing its name with a special character, as shown in the table below.

| Prefix | Example | Object Created |
| :-: | :- | :- |
| none | `)ed new` | Function | 
| `∇` | `)ed ∇ borscht` | Function |
| `-` | `)ed - papyrus` | Simple character matrix |
| `→` | `)ed → crouton` | Simple character vector |
| `∊` | `)ed ∊ grunt` | Nested vector of character vectors, with one sub-vector per line |

See also [the appendix](./Appendices.ipynb#Invoking-the-Editor) for additional prefixes.

It is possible to open several edit windows using a single command. For example, `)ed Tyrex -moose` will open two edit windows. The first to create or edit a function named `Tyrex` and the second to create a character matrix named `moose`.

If a prefix is specified for the name of an already existing object, the prefix is ignored and the editor is invoked according to the type of the existing object.

There are some other ways to invoke the editor:

 - use `⎕ED` instead of the command `)ED`. For example: `⎕ED 'Clown'`. `⎕ED` is a *System function*, a concept that will be discussed in [a future chapter](./System-Interfaces.ipynb);
 - type a name, or put the input cursor on an existing name, and activate the menu "Action" &#8680; "Edit";
 - for the Windows interpreter, type a name or put the input cursor on an existing name, and click the "Edit Object" available in the toolbar (cf. <!--figure-->the image below<!--Win_Toolbar_Edit_Object-->).
 
![The "Edit Object" button in the toolbar of the Windows interpreter](res/Win_Toolbar_Edit_Object.png)

### What Can You Edit?

The following table shows the different item types that can be displayed using the text editor. Some of them can be modified (are "Editable") while others cannot: they can only be viewed.

The table contains a list of the item types, the default foreground and background colours used to represent them and a "Yes" or "No" depending on whether they can be modified or not.

| Item type | Foreground | Background | Editable |
| :- | :-: | :-: | :-: |
| Function | *various* | White | Yes |
| Character matrix | Green | Black | Yes |
| Character vector | Black | White | Yes |
| Vector of character vectors | Blue | Black | Yes |
| Any numeric array | White | Grey | No |
| Mixed array | Blue | Grey | No |
| Object representation (`⎕OR`) | White | Red | No |

### What Can You Do?

#### Cut / Copy / Paste

it is possible to Cut / Copy / Paste text inside an edit window but also from one edit window to another one. It is also possible to copy text from the session window and paste it into any function, operator or editable variable, or vice-versa. So, if you have entered some experimental expressions in the session you can drag and drop them into a defined function.

On Windows, if you installed the default keyboard layout that comes with Dyalog APL, it may happen that <kbd>Ctrl</kbd>+<kbd>c</kbd> produces an APL glyph instead of copying the selected text. If this is the case, you can either install a layout that doesn't have these collisions, like [abrudz's AltGr layout](https://github.com/abrudz/kbd), or you can right-click and then select Cut / Copy / Paste, instead of using the usual keyboard shortcuts.

#### Drag/Drop Restrictions

You may *move* or *copy* test using drag/drop but the following behaviour applies:

 - if you drag/drop within the same Edit window, the default operation is a *move*. If you press the <kbd>Ctrl</kbd> key at the same time, the operation is a *copy*;
 - if you drag/drop text from one window to another window, the operation is always a *copy*;
 - if you drag/drop text within the Session window, the operation is always a *copy*.

#### Exit the Editor

We have already seen the main ways of leaving the editor:

 - press the <kbd>Esc</kbd> key (or use "File" &#8680; "Exit (and Fix)" on the Windows editor). This fixes the modifications and closes the edit window;
 - press <kbd>Shift</kbd>+<kbd>Esc</kbd> (or use "File &#8680; "Exit and discard changes" on the Windows editor). This leaves the editor without saving modifications; they are lost;
 - on Windows, "File" &#8680; "Fix" fixes the modifications but does not close the edit window.

### Undo, Redo, Replay

#### Undo

As long as the contents of an Edit window have not been fixed, it is possible to undo all the modifications made since the last fix.

To ***undo*** modifications you can

 - press <kbd>Ctrl</kbd>+<kbd>Shift</kbd>+<kbd>Backspace</kbd> as many times as needed, or
 - activate the menu "Edit" &#8680; "Undo".
 
If your APL keyboard layout allows it, you can also use <kbd>Ctrl</kbd>+<kbd>z</kbd>.

#### Redo

Having used ***undo***, it is possible to restore the changes that you have undone, one by one.

To ***redo*** modifications, you can

 - press <kbd>Ctrl</kbd>+<kbd>Shift</kbd>+<kbd>Enter</kbd> as many times as needed, or
 - activate the menu "Edit" &#8680; "Redo".
 
If you APL keyboard layout allows it, you can also use <kbd>Ctrl</kbd>+<kbd>y</kbd>.

It must be emphasised that the *Undo/Redo* facility applies only to the current window. If some pieces of text have been copied to another window, the contents of the other window are not affected by the *Undo* or *Redo* operations.

#### Replay Input Lines

The same keyboard shortcuts (<kbd>Ctrl</kbd>+<kbd>Shift</kbd>+<kbd>Backspace</kbd> and <kbd>Ctrl</kbd>+<kbd>Shift</kbd>+<kbd>Enter</kbd>) can be used to scroll up and down the statements that you have previously entered into the session window. For example, suppose you have made several sequential calculations that generate a considerable amount of output, only then to realise you made a mistake in the beginning.

You *could* scroll back through the considerable amount of output to find the statement with the error. However, you can search just through the *input* statements instead (only the lines that you have entered, excluding any output), which are stored in a dedicated Input History buffer:

 - press <kbd>Ctrl</kbd>+<kbd>Shift</kbd>+<kbd>Backspace</kbd> to scroll back through the Input History Buffer as many times as needed;
 - press <kbd>Ctrl</kbd>+<kbd>Shift</kbd>+<kbd>Enter</kbd> to scroll forward if you went too far back.

Once you have found the line, you can change it (or not) and then execute it again by pressing the <kbd>Enter</kbd> key.

The size of this dedicated buffer is controlled by the following configuration parameter on the Windows interpreter:

<center>"Options" &#8680; "Configure..." &#8680; "Session" &#8680; "History buffer size"</center>

WHAT ABOUT RIDE?

## Miscellaneous


### List of Variables and Functions

You can obtain a list of your variables by typing

In [263]:
)Vars

FILEPATH Val a actual b c charMat chemistry d dream dreamLines 
file1 file2 filepath forecast hours inverses mat nesMat nm number salaries
times totMat towns u v var which x zap

and you can obtain a list of your functions by typing

In [264]:
)Fns

AMHM AbsVal Add3 Average BadSyntaxTradfn CasePotpourri CheckVar CollatzPath Comment 
ComputeGrowthTime ContinueInsideWhile Count Dispatch DivideIt Divisors 
DivisorsNoLoops Endless Extract Fact Fact2 HarmonicMean Hypo Ignore Interlace IsItExpensive 
IsItExpensiveDfn Lengths Magic MultipleDestinations MultiplyBy10 NestedRes NewName Plus 
PlusNoRes PrintAB PrintCD PrintE Prompt1 Prompt2 QuadRoot Reverse Rho Root RootOf10 
Round SaneIsItExpensive SetAThenPrint SetBThenPrint SetCThenPrint SetDThenPrint SetLocalE 
ShyFun Sign Silly Sum Switch1 Switch2 Times Times2 TimesTwo To TradHarmonicMean 
Useless VarLevels ZapMe

### Use of the Result

To sum up what we have already seen, once a function has been defined and we call it, its result can be:

 - Immediately displayed and lost:

In [265]:
HarmonicMean times

0.895522

 - Included in an expression:

In [266]:
60×HarmonicMean times

53.7313

 - Assigned to a variable:

In [267]:
hours ← HarmonicMean times

### Vector Representation

We saw that double-clicking on a function name invokes the editor and allows the user to edit the code. We can also type the name of the function and see its code:

In [268]:
TradHarmonicMean

∇ mean←TradHarmonicMean argVector;inverses
[1] inverses←÷argVector 
[2] mean←÷+/inverses 
 ∇

One can also obtain this representation (as a character array) using the built-in ***System function*** `⎕VR` (for "*Vector Representation*") of Dyalog APL. *System functions* are a special kind of function provided with the development environment. The first character of their name is a ***Quad*** (`⎕`) which guarantees that they cannot conflict with user-defined names, and their names are also case-insensitive:

In [269]:
⎕VR 'TradHarmonicMean'

∇ mean←TradHarmonicMean argVector;inverses
[1] inverses←÷argVector 
[2] mean←÷+/inverses 
 ∇

In [270]:
⎕Vr 'TradHarmonicMean'

∇ mean←TradHarmonicMean argVector;inverses
[1] inverses←÷argVector 
[2] mean←÷+/inverses 
 ∇

In [271]:
⎕vr 'TradHarmonicMean'

∇ mean←TradHarmonicMean argVector;inverses
[1] inverses←÷argVector 
[2] mean←÷+/inverses 
 ∇

In [272]:
⎕vR 'TradHarmonicMean'

∇ mean←TradHarmonicMean argVector;inverses
[1] inverses←÷argVector 
[2] mean←÷+/inverses 
 ∇

Note that this is quite unusual in a programming language. The result of `⎕VR` is a character vector representing the source code of our function, which is now available for processing by other functions in the workspace!

System functions will be discussed in detail in [a later chapter](./System-Interfaces.ipynb).

### Source Code Management

Up to now, all the functions and operators we defined were created with an APL code editor and stored in an APL workspace.

This monolithic *one-workspace* approach to application development works well for small applications developed by single programmers, but is often inappropriate for large applications developed by single programmers. Some groups have tacked this problem by storing code in ancillary workspaces or special files. Code is then copied dynamically into the main workspace when required, using techniques that will be explained later.

Had the same application been developed in a more traditional language, programs would be entered and modified using a text editor, stored in separate text files, and maintained independently from one another under the aegis of a *source code management* system that allows the team of programmers to coordinate their activities and keep track of changes.

This type of application development approach is also available in Dyalog APL, using source code management systems like ***SALT*** (for *Simple APL Library Toolkit*) and ***Link***.

These systems make it possible to store sets of APL functions, operators and variables in text files that may be edited and managed using either the built-in APL code editor or industry standard tools This makes it easier to share code between projects and teams of developers.

To take full advantage of this new technique of developing applications, you should first learn about *Namespaces* and related topics, notations and commands. For this reason, these tools will be studied later in [the chapter about source code management](./Source-Code-Management.ipynb).

![Man drowning in APL glyphs is reassured by another man telling him about the "First Aid Kit" chapter.](res/Drowning.png)

## Exercises on Tradfns

**Exercise 11**:

Write a function which displays the greatest value in a numeric matrix, along with its position (row and column) in the matrix, for example:

In [347]:
∇ msg ← MaxPlace matrix
∇

In [348]:
MaxPlace actual
⍝ The greatest value: 519, is in row 4, column 4

**Exercise 12**:

Conversions from Celsius to Fahrenheit degrees and back can be done using the following formulas:

 - `F ← 32+9×C÷5` converts from Celsius to Fahrenheit;
 - `C ← 5×(F-32)÷9` converts from Fahrenheit to Celsius.
 
 Can you program a function that makes the conversion `C→F` or `F→C` according to its right argument?

In [349]:
∇ converted ← temps Convert conversion
∇

In [350]:
86 32 212 Convert 'F-C' ⍝ 30 0 100
7 15 25   Convert 'C-F' ⍝ 44.6 59 7

**Exercise 13**:

Summing the items of a vector is so simple in APL (`+/vec`) that one cannot understand why this simple problem needs a loop in traditional languages! Just for fun, can you program such a loop in APL? Use control structures.

In [351]:
+/ 31 37 44 19 27 60 42

260

In [352]:
∇ sum ← LoopSum vec
∇

In [353]:
LoopSum 31 37 44 19 27 60 42 ⍝ 260

**Exercise 14:**

In exercise 3 you were asked to reverse the order of a vector of items. Even if it is a strange idea, can you do the same operation using a loop, moving letter after letter?

In [354]:
∇ reversed ← ReverLoop string
∇

In [355]:
ReverLoop 'The solution without loop was much better'
⍝ retteb hcum saw pool tuohtiw noitulos ehT

**Exercise 15**:

In a given numeric matrix with $n$ columns, we would like to insert subtotals after each group of $g$ columns (where $g$ is a divisor of $n$).

Try to write a function to do that, following these 3 steps:

 1. Reshape the matrix so that it fits in $g$ columns only, with the necessary number of rows to contain all the values.
 2. Concatenate, on the right, the totals of each row.
 3. Reshape again that new matrix to obtain the final result.
 
In the examples below, the original matrix appears in black and the subtotals appear in grey:

In [358]:
actual

89 166 420 508 12 23
111 453 177 365 284 352
349 192 329 115 515 374
160 467 234 519 485 283

`3 SubSum actual` should give

$$
\begin{matrix}
 89 & 166 & 420 &\color{grey}{675} & 508 & 12 &  23 &\color{grey}{543} \\
111 & 453 & 177 &\color{grey}{741} & 365 & 284 & 352 &\color{grey}{1001} \\
349 & 192 & 329 &\color{grey}{870} & 115 & 515 & 374 &\color{grey}{1004} \\
160 & 467 & 234 &\color{grey}{861} & 519 & 485 & 283 &\color{grey}{1287}  
\end{matrix}
$$

and `2 SubSum actual` should give

$$
\begin{matrix}
 89 & 166 &\color{grey}{255} & 420 & 508 &\color{grey}{928} & 12 &  23 &\color{grey}{35} \\
111 & 453 &\color{grey}{564} & 177 & 365 &\color{grey}{542} & 284 & 352 &\color{grey}{636} \\
349 & 192 &\color{grey}{541} & 329 & 115 &\color{grey}{444} & 515 & 374 &\color{grey}{889} \\
160 & 467 &\color{grey}{627} & 234 & 519 &\color{grey}{753} & 485 & 283 &\color{grey}{768}  
\end{matrix}
$$

In [359]:
∇ sub ← g SubSum mat
    (r c) ← ⍴mat
    rows ← (r×c)÷g
    reshaped ← rows g⍴mat
    reshaped ← reshaped,+/reshaped
    newc ← c+c÷g
    sub ← r newc⍴reshaped
∇

In [361]:
3 SubSum actual

89 166 420 675 508 12 23 543
111 453 177 741 365 284 352 1001
349 192 329 870 115 515 374 1004
160 467 234 861 519 485 283 1287

In [362]:
2 SubSum actual

89 166 255 420 508 928 12 23 35
111 453 564 177 365 542 284 352 636
349 192 541 329 115 444 515 374 889
160 467 627 234 519 753 485 283 768

**Exercise 16**:

This is a very classic problem: we want to partition a text vector each time a given separator is found and make a matrix from these pieces. We will see later a solution without a loop, but for the moment, sorry, you will need a loop.

In [363]:
∇ mat ← sep Sorry string
∇

In [364]:
'/' Sorry 'You/will/need a/loop/to solve/this/exercise'
⍝ You
⍝ will
⍝ need a
⍝ loop
⍝ to solve
⍝ this
⍝ exercise

**Exercise 17**:

Remember when we talked about the ["Collatz Conjecture"](https://en.wikipedia.org/wiki/Collatz_conjecture) earlier on? Don't worry if you don't. Now you will implement a function that takes a positive integer `n` and creates a vector of numbers, by successively taking the most recent number and then:

 - if the most recent number is **even**, the next value will be half of it;
 - if the most recent number is **odd**, the next value will be 1 more than the triple of it;
 - stop when you reach 1.
 
For example, starting with `5` you get `5 16 8 4 2 1`.

It is believed that no matter what initial positive integer `n` you give it, this process always stops at 1.

In [365]:
∇ path ← CollatzPath n
∇

In [366]:
CollatzPath 37
⍝ 37 112 56 28 14 7 22 11 34 17 52 26 13 40 20 10 5 16 8 4 2 1

### Exercise Variants

Now that you were given some exercises on dfns (cf. [this](#Exercises-on-Dfns)) and some exercises on tradfns (cf. [this](#Exercises-on-Tradfns)), which ones can you solve with the other method? Solving exercises 11 to 17 with dfns will be harder than solving exercises 1 to 10 with tradfns.

In fact, for some of these exercises you may need to learn some more primitives before you can change the tradfns to dfns that do the same thing.

Also, can you solve exercise 17 with a recursive function? Either dfn or tradfn, your choice.

## Solutions

**Exercise 1**:

If `⍵` is the vector argument, we can use `⍵[...]` to index into `⍵` and then we can use the *Index Generator* primitive to generate the indices we need, which should be the integers from `1` to `⍺`... Except that if `⍺` is too big, we cannot generate indices larger than the length of the vector, so we also find the minimum between `⍺` and `≢⍵`. If we don't, we get a `INDEX ERROR` when indexing. Here is a possible implementation:

In [273]:
]dinput
Extract ← {
    ⍵[⍳⍺⌊≢⍵]
}

In [274]:
3 Extract 45 86 31 20 75 62 18

45 86 31

In [275]:
6 Extract 'can you do it?'

can yo

In [276]:
20 Extract 1 2 3

1 2 3

**Exercise 2**:

We can use a similar logic as that of the first exercise, except now we want to start the indices at `⍺+1` and go up until `≢⍵`. For this to happen, we first need to find out how many numbers we need. If a vector has `≢⍵` elements and we are going to drop `⍺` of them, we are going to be left with `(≢⍵)-⍺`. This means `⍳(≢⍵)-⍺` will generate the correct amount of indices, but they will be starting at `1` and should start at `⍺+1`, so we just need to add `⍺` to that.

Finally, we just need to worry about what happens if `⍺` is too large, i.e. if we want to ignore too many elements. The reverse of that concern is, what happens if `(≢⍵)-⍺` is too small? Recall that `(≢⍵)-⍺` tells you how many elements you will want to keep. But that number must be at least `0` elements (i.e. "keep no elements") because it makes no sense to keep a negative number of elements. So we can just use `⌈` to find the maximum between `0` and `(≢⍵)-⍺`. If `⍺` is too large, `0⌈(≢⍵)-⍺` gives `0` and `⍳0` is the empty vector `⍬`, so the indexing will work just fine.

In [277]:
]dinput
Ignore ← {
    ⍵[⍺+⍳0⌈(≢⍵)-⍺]
}

In [278]:
3 Ignore 45 86 31 20 75 62 18

20 75 62 18

In [279]:
6 Ignore 'can you do it?'

u do it?

In [280]:
20 Ignore 1 2 3

**Exercise 3**:

Here is another exercise on index arithmetic. Here is what we want to happen with a vector argument of length 10:

 - generate the indices `1 2 3 4 5 6 7 8 9 10`
 - transform them into  `10 9 8 7 6 5 4 3 2 1`
 
We can do this if we do the correct subtraction:

In [281]:
11 - ⍳10 

10 9 8 7 6 5 4 3 2 1

But here `11` was a special number: it was `1+≢⍵`. So that is the general tactic we must employ:

In [282]:
]dinput
Reverse ← {
    ⍵[(1+≢⍵)-⍳≢⍵]
}

In [283]:
Reverse 'snoitalutargnoc'

congratulations

In [284]:
Reverse '!ti did uoY'

You did it!

**Exercise 4**:

This exercise can be solved by using the *Reduce* operator to sum: `+/`. Then we need to specify the axis we care about with `[1]` and `[2]`.

If we do `+/[1]` then we are reducing across the first axis, which means we get the sums along the columns:

In [285]:
⊢mat ← 3 4⍴75 14 86 20 31 16 40 51 22 64 31 28

75 14 86 20
31 16 40 51
22 64 31 28

In [286]:
+/[1]mat

128 94 157 99

We can then catenate the original matrix to these column sums vertically (by using `⍪`), and then use `+/[2]` to find the row sums and catenate them with `,`:

In [287]:
]dinput
Totalise ← {
    colSums ← +/[1]⍵
    r ← ⍵⍪colSums
    rowSums ← +/[2]r
    r,rowSums
}

In [288]:
Totalise mat

75 14 86 20 195
 31 16 40 51 138
 22 64 31 28 145
128 94 157 99 478

In [289]:
totMat ← 4 5⍴75 14 86 20 195 31 16 40 51 138 22 64 31 28 145 128 94 157 99 478
totMat ≡ Totalise mat

1

**Exercise 5**:

When reading this exercise, one should immediately realise that one is going to need to find *Where* the blank spaces are:

In [290]:
text ← 'This seems to be a good solution'

In [291]:
⍸' '=text

5 11 14 17 19 24

These indices tell where the blank spaces were in the character vector, and in between those indices are the indices that correspond to word characters:

 - the first word has indices `1 2 3 4`
   - then there is a space at position `5`
 - the second word has indices `6 7 8 9 10`
   - then there is a space at position `11`
 - ...
   - then there is a space at position `24`
 - the last word has indices `25 26 27 28 29 30 31 32`
 
The `32` above is `≢text`:

In [292]:
≢text

32

From the list above we can see that most words are *between* spaces, but the first and last words may not be between spaces. We can fix this by *forcing* the first and last words to be *between* spaces if we had a single `' '` to the beginning and to the end of our variable:

In [293]:
⍸' '=' ',text,' '

1 6 12 15 18 20 25 34

Now we have

 - the first space at position `1`
   - the first word in positions `2 3 4 5`
 - a space at position `6`
   - a word in positions `7 8 9 10 11`
 - ...
 - a space at position `25`
   - the last word in positions `26 27 28 29 30 31 32 33`
 - the final space at position `34`
 
So we can find the lengths of those runs of non-spaces by subtracting positions of consecutive spaces and then subtracting 1 from those, because `6-1` gives 5, but between 1 and 6 there's only 4 integers.

In [294]:
]dinput
Lengths ← {
    spaces ← ⍸' '=' ',⍵,' '
    idx ← ⍳(≢spaces)-1
    ¯1+spaces[1+idx]-spaces[idx]
}

In [295]:
Lengths 'This seems to be a good solution'

4 5 2 2 1 4 8

The final step where we index into `spaces` to get "all but the last" and "all but the first" elements of `spaces` could have been done with your previous solutions:

In [296]:
]dinput
Lengths ← {
    spaces ← ⍸' '=' ',⍵,' '
    ¯1+(1 Ignore spaces)-((¯1+≢spaces) Extract spaces)
}

In [297]:
Lengths 'This seems to be a good solution'

4 5 2 2 1 4 8

Notice that doing `¯1+expr` is a little "trick" you can employ when you want to subtract 1 from `expr`, but `expr` would then need parenthesis if you were to have it on the left of the `-` sign. For example, to subtract 1 from `≢spaces` you would have to do `(≢spaces)-1` but instead you can do `¯1+≢spaces`.

Finally, can you improve your solution to handle multiple consecutive spaces?

In [298]:
Lengths 'This only    has      five words   '

4 4 0 0 0 3 0 0 0 0 0 4 5 0 0 0

Probably seeing how your function works with multiple consecutive spaces gives the solution away: consecutive spaces will make a 0 appear in the final result, so we just have to remove those:

In [299]:
]dinput
Lengths ← {
    spaces ← ⍸' '=' ',⍵,' '
    r ← ¯1+(1 Ignore spaces)-((¯1+≢spaces) Extract spaces)
    (0≠r)/r
}

In [300]:
Lengths 'This only    has      five words   '

4 4 3 4 5

**Exercise 6**:

We have seen in [a previous section](./Some-Primitive-Functions.ipynb#Basic-Usage) how to create any arithmetic sequence of integers. This is just a special case of the algorithm give, with `Step ← 1`:

In [301]:
]dinput
To ← {
    ⍺+¯1+⍳(1+⍵-⍺)
}

In [302]:
17 To 29

17 18 19 20 21 22 23 24 25 26 27 28 29

**Exercise 7**:

This exercise is easier than it might look because the primitives to catenate vertically and horizontally, `⍪` and `,`, know how to deal with a matrix and a single scalar:

In [303]:
⊢towns ← 6 10⍴'Canberra  Paris     WashingtonMoscow    Martigues Mexico    '

Canberra 
Paris 
Washington
Moscow 
Martigues 
Mexico

In [304]:
towns,'|'

Canberra |
Paris |
Washington|
Moscow |
Martigues |
Mexico |

In [305]:
towns⍪'-'

Canberra 
Paris 
Washington
Moscow 
Martigues 
Mexico 
----------

So we just have to frame the four sides and *then* change the corners:

In [306]:
]dinput
Frame ← {
    f ← '|',⍵,'|'
    f ← '-'⍪f⍪'-'
    (r c) ← ⍴f
    f[1 r;1 c] ← '+'
    f
}

In [307]:
Frame towns

+----------+
|Canberra |
|Paris |
|Washington|
|Moscow |
|Martigues |
|Mexico |
+----------+

Here we used the very convenient indexing notation `f[1 r;1 c]` that allows us to access the positions `1 1`, `1 c`, `r 1` and `r c` of the matrix `f`.

Modifying this function to use the appropriate line-drawing symbols just means swapping the `'|-+'` in the original function. Care must be taken, however, when assigning the corners. With `f[1 r;1 c] ← m` APL expects `m` to be a scalar *or* an array with the same shape as that of the left, and since `f[1 r;1 c]` is a 2 by 2 matrix, we will have to reshape the vector with the corners into a 2 by 2 matrix as well:

In [308]:
]dinput
Frame ← {
    f ← (⎕UCS 9474) , ⍵ , (⎕UCS 9474)
    f ← (⎕UCS 9472) ⍪ f ⍪ (⎕UCS 9472)
    (r c) ← ⍴f
    f[1 r;1 c] ← 2 2⍴⎕UCS 9484 9488 9492 9496
    f
}

In [309]:
Frame towns

┌──────────┐
│Canberra │
│Paris │
│Washington│
│Moscow │
│Martigues │
│Mexico │
└──────────┘

**Exercise 8**:

Well, what if what we wrote actually works for vectors? Let's give it a try:

In [310]:
Frame 'We are not out of the wood'

RANK ERROR
Frame[4] f[1 r;1 c]←2 2⍴⎕UCS 9484 9488 9492 9496
          ∧


A `RANK ERROR`? That makes no sense, after I frame `⍵` with the horizontal and vertical bars I have a framed matrix, I just need to update the corners... right? Wrong! Here's what happens if you use `,` and `⍪` on a vector:

In [311]:
text ← 'We are not out of the wood'
(⎕UCS 9472) ⍪ (⎕UCS 9474) , text , (⎕UCS 9474) ⍪ (⎕UCS 9472)

─│We are not out of the wood│─

Because `text` has shape

In [312]:
⍴text

26

the primitives `,` and `⍪` work the same way. We need to turn input vectors into matrices with 1 row before we proceed with the framing process.

Let us define `shape ← ⍴⍵` as the shape of the input `Frame` gets. If `⍵` is a matrix, then `shape` is the appropriate shape, otherwise we need `⍵` to be reshaped into `1,shape`. In traditional programming languages we could use an `if-else` statement. However, dfns do not have support for such flow control structures and so we need to handle this matter in a different way.

A possibility is to create the vector `v←1,shape` and then index into it with care. If `⍵` is a vector, `v` has 2 elements and we want both. If `⍵` is a matrix, `v` has 3 elements and we want the elements in positions `2 3`. A way of generating the indices `1 2` if `⍵` is a vector and `2 3` if `⍵` is a matrix is with the expression `0 1+≢⍴⍵`:

| `⍵` | `≢⍴⍵` | `0 1+≢⍴⍵` |
| :- | :-: | :-: |
| vector | 1 | 1 2 |
| matrix | 2 | 2 3 |

When implementing the function we don't need to actually create `v`:

In [313]:
]dinput
Frame ← {
    shape ← (1,⍴⍵)[0 1+≢⍴⍵]
    f ← shape⍴⍵
    f ← (⎕UCS 9474) , f , (⎕UCS 9474)
    f ← (⎕UCS 9472) ⍪ f ⍪ (⎕UCS 9472)
    (r c) ← ⍴f
    f[1 r;1 c] ← 2 2⍴⎕UCS 9484 9488 9492 9496
    f
}

In [314]:
Frame text

┌──────────────────────────┐
│We are not out of the wood│
└──────────────────────────┘

In [315]:
Frame towns

┌──────────┐
│Canberra │
│Paris │
│Washington│
│Moscow │
│Martigues │
│Mexico │
└──────────┘

**Exercise 9**:

The logic to solving this task resembles what we did in the first exercises. First we will find *Where* the first letter is, and then we will use indexing to put the second letter in those positions:

In [316]:
]dinput
Switch1 ← {
    r ← ⍵
    r[⍸⍺[1]=⍵] ← ⍺[2]
    r
}

In [317]:
'tc' Switch1 'A bird in the hand is worth two in the bush'

A bird in che hand is worch cwo in che bush

We take the intermediate step of doing `r ← ⍵` because we can't assign to `⍵` and so `⍵[⍸⍺[1]=⍵] ← ⍺[2]` wouldn't work.

**Exercise 10**:

A very obvious modification of the function above is to write

In [318]:
]dinput
Switch2 ← {
    r ← ⍵
    r[⍸⍺[1]=⍵] ← ⍺[2]
    r[⍸⍺[2]=⍵] ← ⍺[1]
    r
}

In [319]:
'ei' Switch2 'A bird in the hand is worth two in the bush'

A berd en thi hand es worth two en thi bush

However, a really elegant solution becomes possible if we use the *Index Of* primitive and the concept of changing the frame of reference we discussed previously (cf. [Changing The Frame of Reference](./Some-Primitive-Functions.ipynb#Changing-The-Frame-of-Reference)). We used this concept to convert lower case letters into upper case letters.

In [320]:
]dinput
Switch2 ← {
    pos ← (⍺,⍵)⍳⍵
    (⍺[2 1],⍵)[pos]
}

In [321]:
'ei' Switch2 'A bird in the hand is worth two in the bush'

A berd en thi hand es worth two en thi bush

What exactly is happening? Well, we are basically establishing the initial and final sets (as seen in [Changing The Frame of Reference](./Some-Primitive-Functions.ipynb#Changing-The-Frame-of-Reference)) as the sentence itself, but preceded by the two characters. For the initial set, we have them in their input order (the `(⍺,⍵)` above) but for the final set we swap them (the `⍺[2 1],⍵` above).

This establishes the following "conversion":

```
eiA bird in the hand is worth two in the bush
ieA bird in the hand is worth two in the bush
```

Whenever I have a character, I look for it in the first line, stopping as soon as I find it (that is what `⍳` does) and then I swap it with the corresponding character in the line below.

We can thus re-implement `Switch2` with more intermediate steps, to make this more obvious:

In [322]:
]dinput
Switch2 ← {
    ⎕← initialSet ← ⍺,⍵
    pos ← initialSet⍳⍵
    ⎕← finalSet ← ⍺[2 1],⍵
    finalSet[pos]
}

In [323]:
'ei' Switch2 'A bird in the hand is worth two in the bush'

eiA bird in the hand is worth two in the bush
ieA bird in the hand is worth two in the bush
A berd en thi hand es worth two en thi bush